In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/soccer'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../input/soccer/database.sqlite


In [5]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///../input/soccer/database.sqlite")
engine.table_names()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  This is separate from the ipykernel package so we can avoid doing imports until


['Country',
 'League',
 'Match',
 'Player',
 'Player_Attributes',
 'Team',
 'Team_Attributes',
 'sqlite_sequence']

### **Number of Matches played in the 11 leagues**

In [6]:
pd.read_sql_query('''SELECT name, COUNT(country_id) AS matches
                     FROM league
                     LEFT JOIN Match
                     USING(country_id)
                     GROUP BY name
                     LIMIT 5;''', engine)

,name,matches
0,Belgium Jupiler League,1728
1,England Premier League,3040
2,France Ligue 1,3040
3,Germany 1. Bundesliga,2448
4,Italy Serie A,3017


Suppose we want to compare the number of 
* home team wins, 
* away team wins,
* ties

In [7]:
pd.read_sql_query('''SELECT date, id, home_team_goal, away_team_goal
                     FROM Match 
                     WHERE season = '2013/2014'
                     AND home_team_goal > away_team_goal
                     LIMIT 4;  ''', engine)

,date,id,home_team_goal,away_team_goal
0,2014-03-29 00:00:00,1237,2,0
1,2014-04-05 00:00:00,1239,1,0
2,2014-04-12 00:00:00,1241,2,1
3,2014-04-12 00:00:00,1242,2,0


This is not efficient so we will use
# CASE statements

* `WHEN`, `THEN`, `ELSE`, `END`

In [8]:
pd.read_sql_query('''SELECT id, home_team_goal, away_team_goal,
                     CASE WHEN home_team_goal > away_team_goal THEN 'Home Team Win'
                          WHEN home_team_goal < away_team_goal THEN 'Away Team Win'
                          ELSE 'Tie' END AS outcome
                     FROM Match 
                     WHERE season = '2013/2014'
                     LIMIT 4;  ''', engine)

,id,home_team_goal,away_team_goal,outcome
0,1237,2,0,Home Team Win
1,1238,0,1,Away Team Win
2,1239,1,0,Home Team Win
3,1240,0,0,Tie


### **Basic CASE statements**
What is your favorite team?

The *European Soccer Database* contains data about 12,800 matches from 11 countries played between 2011-2015! Throughout this course, you will be shown filtered versions of the tables in this database in order to better explore their contents.

In this exercise, you will identify matches played between *FC Schalke 04* and *FC Bayern Munich*. There are 2 teams identified in each match in the `hometeam_id` and `awayteam_id` columns, available to you in the `Match` table. ID can join to the `team_api_id` column in the `Team` table, but you cannot perform a join on both at the same time.

However, you can perform this operation using a `CASE` statement once you've identified the `team_api_id` associated with each team!

* Select the team's long name and API id from the `Team` table.
* Filter the query for *FC Schalke 04* and *FC Bayern Munich* using `IN`, giving you the `team_api_id`s needed for the next step.

In [9]:
pd.read_sql_query('''SELECT team_long_name,
                            team_api_id
                     FROM Team
                     WHERE team_long_name IN ('FC Schalke 04', 'FC Bayern Munich');''', engine)

,team_long_name,team_api_id
0,FC Bayern Munich,9823
1,FC Schalke 04,10189


* Create a `CASE` statement that identifies whether a match in Germany included `FC Bayern Munich`, `FC Schalke 04`, or neither as the home team.
* Group the query by the `CASE` statement alias, `home_team`.

In [10]:
pd.read_sql_query('''SELECT CASE WHEN home_team_api_id = 10189 THEN 'FC Schalke 04'
                                 WHEN away_team_api_id = 9823 THEN 'FC Bayern Munich'
                                 ELSE 'Other' END AS home_team,
                            COUNT(id) AS total_matches
                      FROM Match
                      GROUP BY home_team; ''', engine)

,home_team,total_matches
0,FC Bayern Munich,128
1,FC Schalke 04,136
2,Other,25715


### **CASE statements comparing column values**

Barcelona is considered one of the strongest teams in Spain's soccer league.

In this exercise, you will be creating a list of matches in the 2011/2012 season where Barcelona was the home team. You will do this using a `CASE` statement that compares the values of two columns to create a new group -- wins, losses, and ties.

In 3 steps, you will build a query that identifies a match's winner, identifies the identity of the opponent, and finally filters for Barcelona as the home team. Completing a query in this order will allow you to watch your results take shape with each new piece of information.

The `Match` table currently contains  matches from the 2011/2012 season, and has two key columns, `home_team_api_id` and `away_team_api_id`, that can be joined with the `Team` table. However, you can only join `Team` to one column at a time.

* Select the `date` of the match and create a `CASE` statement to identify matches as home wins, home losses, or ties.

In [11]:
pd.read_sql_query('''SELECT date,
                            CASE WHEN home_team_goal > away_team_goal THEN 'Home win!'
                                 WHEN home_team_goal < away_team_goal THEN 'Home loss :(' 
                                 ELSE 'Tie' END AS outcome
                     FROM Match
                     WHERE season = '2011/2012'
                     LIMIT 5;''', engine)

,date,outcome
0,2011-07-29 00:00:00,Home win!
1,2011-07-30 00:00:00,Tie
2,2011-07-30 00:00:00,Home win!
3,2011-07-30 00:00:00,Home loss :(
4,2011-07-30 00:00:00,Tie


* Left join the `Team` table `team_api_id` column to the `Match` table `away_team_api_id`. This allows us to retrieve the *away* team's identity.
* Select `team_long_name` from `Team` as `opponent` and complete the `CASE` statement from Step 1.

In [12]:
pd.read_sql_query('''SELECT date,
	                        team_long_name AS opponent, 
	                        CASE WHEN home_team_goal > away_team_goal THEN 'Home win!'
                                 WHEN home_team_goal < away_team_goal THEN 'Home loss :('
                                 ELSE 'Tie' END AS outcome
                      FROM Match 
-- Left join Team onto Match
                      LEFT JOIN Team 
                      ON away_team_api_id = team_api_id
                      WHERE season = '2011/2012' LIMIT 4;''', engine)

,date,opponent,outcome
0,2011-07-29 00:00:00,RSC Anderlecht,Home win!
1,2011-07-30 00:00:00,Standard de Liège,Tie
2,2011-07-30 00:00:00,Beerschot AC,Home win!
3,2011-07-30 00:00:00,KSV Cercle Brugge,Home loss :(


* Modify the same `CASE` statement as the previous steps.
* Filter for matches where the home team is FC Barcelona (id = `8634`).

In [13]:
pd.read_sql_query('''SELECT date,
	                        team_long_name AS opponent, 
	                        CASE WHEN home_team_goal > away_team_goal THEN 'Barcelona win!'
                                 WHEN home_team_goal < away_team_goal THEN 'Barcelona loss :('
                                 ELSE 'Tie' END AS outcome
                      FROM Match
                      LEFT JOIN Team
                      ON away_team_api_id = team_api_id
                      WHERE season = '2011/2012'
                      AND home_team_api_id = 8634;''', engine)

,date,opponent,outcome
0,2011-10-29 00:00:00,RCD Mallorca,Barcelona win!
1,2011-11-19 00:00:00,Real Zaragoza,Barcelona win!
2,2011-12-03 00:00:00,Levante UD,Barcelona win!
3,2011-11-29 00:00:00,Rayo Vallecano,Barcelona win!
4,2012-01-15 00:00:00,Real Betis Balompié,Barcelona win!
5,2011-08-29 00:00:00,Villarreal CF,Barcelona win!
6,2012-05-02 00:00:00,Málaga CF,Barcelona win!
7,2012-02-04 00:00:00,Real Sociedad,Barcelona win!
8,2012-02-19 00:00:00,Valencia CF,Barcelona win!
9,2012-03-03 00:00:00,Real Sporting de Gijón,Barcelona win!


### **CASE statements comparing two column values part 2**

Similar to the previous exercise, you will construct a query to determine the outcome of Barcelona's matches where they played as the *away team*. You will learn how to combine these two queries soon below.

Did their performance differ from the matches where they were the home team?

* Complete the `CASE` statement to identify Barcelona's away team games (`id = 8634`) as wins, losses, or ties.
* Left join the `Team` table `team_api_id` column on the `Match` table `home_team_id` column. This retrieves the identity of the <ins>*home team*</ins> opponent.
* Filter the query to only include matches where Barcelona was the *away* team.

In [14]:
pd.read_sql_query('''SELECT date, team_long_name AS opponent,
                            CASE WHEN home_team_goal < away_team_goal THEN 'Barcelona win!'
                                 WHEN home_team_goal > away_team_goal THEN 'Barcelona loss :(' 
                                 ELSE 'Tie' END AS outcome
                     FROM Match m
                     LEFT JOIN Team t
                     ON m.home_team_api_id = t.team_api_id
                     WHERE season = '2011/2012'
                     AND m.away_team_api_id = 8634;''', engine)

,date,opponent,outcome
0,2012-01-22 00:00:00,Málaga CF,Barcelona win!
1,2011-10-25 00:00:00,Granada CF,Barcelona win!
2,2011-11-06 00:00:00,Athletic Club de Bilbao,Tie
3,2011-11-26 00:00:00,Getafe CF,Barcelona loss :(
4,2011-12-10 00:00:00,Real Madrid CF,Barcelona win!
5,2012-01-08 00:00:00,RCD Espanyol,Tie
6,2012-01-28 00:00:00,Villarreal CF,Tie
7,2012-02-11 00:00:00,CA Osasuna,Barcelona loss :(
8,2012-02-26 00:00:00,Atlético Madrid,Barcelona win!
9,2012-03-11 00:00:00,Racing Santander,Barcelona win!


## **Reviewing `CASE` `WHEN`**

In [15]:
pd.read_sql_query('''SELECT date, season,
                            CASE WHEN home_team_goal > away_team_goal THEN 'Home team win!'
                                 WHEN home_team_goal < away_team_goal THEN 'Away team win'
                                 ELSE 'Tie' END AS outcome
                            FROM Match 
                            LIMIT 5''', engine)

,date,season,outcome
0,2008-08-17 00:00:00,2008/2009,Tie
1,2008-08-16 00:00:00,2008/2009,Tie
2,2008-08-16 00:00:00,2008/2009,Away team win
3,2008-08-17 00:00:00,2008/2009,Home team win!
4,2008-08-16 00:00:00,2008/2009,Away team win


# **Multiple Logical Conditions**
## **CASE WHEN AND then some ...**

* Use `AND` inside the `WHEN` clause

In [16]:
pd.read_sql_query('''SELECT date,season, home_team_api_id, away_team_api_id,
                            CASE WHEN home_team_goal > away_team_goal AND home_team_api_id = 8455 THEN 'Chelsea home win!'
                                 WHEN home_team_goal < away_team_goal AND away_team_api_id = 8455 THEN 'Chelsea away win'
                                 ELSE 'Loss or Tie :(' END AS outcome
                     FROM Match
                     WHERE home_team_api_id = 8455 OR away_team_api_id = 8455
                     AND season = '2011/2012'
                     LIMIT 10''', engine)

,date,season,home_team_api_id,away_team_api_id,outcome
0,2008-08-17 00:00:00,2008/2009,8455,8462,Chelsea home win!
1,2008-11-01 00:00:00,2008/2009,8455,8472,Chelsea home win!
2,2008-11-22 00:00:00,2008/2009,8455,10261,Loss or Tie :(
3,2008-11-30 00:00:00,2008/2009,8455,9825,Loss or Tie :(
4,2008-12-14 00:00:00,2008/2009,8455,8654,Loss or Tie :(
5,2008-12-26 00:00:00,2008/2009,8455,8659,Chelsea home win!
6,2009-01-17 00:00:00,2008/2009,8455,10194,Chelsea home win!
7,2009-01-28 00:00:00,2008/2009,8455,8549,Chelsea home win!
8,2009-02-07 00:00:00,2008/2009,8455,8667,Loss or Tie :(
9,2009-02-28 00:00:00,2008/2009,8455,8528,Chelsea home win!


ถ้าเอา `WHERE home_team_api_id = 8455` OR `away_team_api_id = 8455`ออก
ก็จะกลายเป็นว่า ผลลัพธ์คู่อื่นๆเป็น แพ้หรือเสมอ กันทั้งหด

In [17]:
pd.read_sql_query('''SELECT date,season, home_team_api_id, away_team_api_id,
                            CASE WHEN home_team_goal > away_team_goal AND home_team_api_id = 8455 THEN 'Chelsea home win!'
                                 WHEN home_team_goal < away_team_goal AND away_team_api_id = 8455 THEN 'Chelsea away win'
                                 ELSE 'Loss or Tie :(' END AS outcome
                     FROM Match
                     WHERE season = '2011/2012'
                     LIMIT 10''', engine)

,date,season,home_team_api_id,away_team_api_id,outcome
0,2011-07-29 00:00:00,2011/2012,1773,8635,Loss or Tie :(
1,2011-07-30 00:00:00,2011/2012,9998,9985,Loss or Tie :(
2,2011-07-30 00:00:00,2011/2012,9987,9993,Loss or Tie :(
3,2011-07-30 00:00:00,2011/2012,9991,9984,Loss or Tie :(
4,2011-07-30 00:00:00,2011/2012,9994,10000,Loss or Tie :(
5,2011-07-30 00:00:00,2011/2012,8571,9989,Loss or Tie :(
6,2011-07-30 00:00:00,2011/2012,8203,9997,Loss or Tie :(
7,2011-07-31 00:00:00,2011/2012,8342,10001,Loss or Tie :(
8,2011-10-16 00:00:00,2011/2012,8342,9991,Loss or Tie :(
9,2011-10-16 00:00:00,2011/2012,8635,9985,Loss or Tie :(


## **FILTER query by CASE**

* ถ้าแค่เอา ELSE ออกไป จะมีค่า NULL โผล่มาเยอะ



In [18]:
pd.read_sql_query('''SELECT date,season, home_team_api_id, away_team_api_id,
                            CASE WHEN home_team_goal > away_team_goal AND home_team_api_id = 8455 THEN 'Chelsea home win!'
                                 WHEN home_team_goal < away_team_goal AND away_team_api_id = 8455 THEN 'Chelsea away win'
                                 END AS outcome
                     FROM Match
                     WHERE home_team_api_id = 8455 OR away_team_api_id = 8455
                     AND season = '2011/2012'
                     LIMIT 10''', engine)

,date,season,home_team_api_id,away_team_api_id,outcome
0,2008-08-17 00:00:00,2008/2009,8455,8462,Chelsea home win!
1,2008-11-01 00:00:00,2008/2009,8455,8472,Chelsea home win!
2,2008-11-22 00:00:00,2008/2009,8455,10261,None
3,2008-11-30 00:00:00,2008/2009,8455,9825,None
4,2008-12-14 00:00:00,2008/2009,8455,8654,None
5,2008-12-26 00:00:00,2008/2009,8455,8659,Chelsea home win!
6,2009-01-17 00:00:00,2008/2009,8455,10194,Chelsea home win!
7,2009-01-28 00:00:00,2008/2009,8455,8549,Chelsea home win!
8,2009-02-07 00:00:00,2008/2009,8455,8667,None
9,2009-02-28 00:00:00,2008/2009,8455,8528,Chelsea home win!


* Include entire CASE without alias in WHERE
* ให้สังเกตว่า ไม่ต้องใส่ `WHERE home_team_api_id = 8455 OR away_team_api_id = 8455`แล้ว

In [19]:
pd.read_sql_query('''SELECT date,season, home_team_api_id, away_team_api_id,
                            CASE WHEN home_team_goal > away_team_goal AND home_team_api_id = 8455 THEN 'Chelsea home win!'
                                 WHEN home_team_goal < away_team_goal AND away_team_api_id = 8455 THEN 'Chelsea away win'
                                 END AS outcome
                     FROM Match
                     WHERE CASE WHEN home_team_goal > away_team_goal AND home_team_api_id = 8455 THEN 'Chelsea home win!'
                                 WHEN home_team_goal < away_team_goal AND away_team_api_id = 8455 THEN 'Chelsea away win'
                                 END IS NOT NULL
                     AND season = '2011/2012'
                     LIMIT 10''', engine)

,date,season,home_team_api_id,away_team_api_id,outcome
0,2011-11-05 00:00:00,2011/2012,8655,8455,Chelsea away win
1,2011-11-26 00:00:00,2011/2012,8455,8602,Chelsea home win!
2,2011-12-03 00:00:00,2011/2012,10261,8455,Chelsea away win
3,2011-12-12 00:00:00,2011/2012,8455,8456,Chelsea home win!
4,2011-08-20 00:00:00,2011/2012,8455,8659,Chelsea home win!
5,2012-01-02 00:00:00,2011/2012,8602,8455,Chelsea away win
6,2012-01-14 00:00:00,2011/2012,8455,8472,Chelsea home win!
7,2012-02-25 00:00:00,2011/2012,8455,8559,Chelsea home win!
8,2012-03-10 00:00:00,2011/2012,8455,10194,Chelsea home win!
9,2011-08-27 00:00:00,2011/2012,8455,9850,Chelsea home win!


## **ก่อนอื่น ลองทำตาราง teams_spain และ matches_spain ตามด้านล่างนี้ก่อน จะได้ไม่ต้องทำจากข้อมูลทั้งหมดเหมือนด้านบน และจะได้ได้ผลลัพธ์ใกล้เคียงกับที่แสดงให้เห็นในแบบฝึกหัดด้วย**

In [20]:
# teams_spain 
# จริงๆต้องเชื่อมด้วยตาราง Match ด้วย Country_id และเชื่อมต่อด้วยตาราง Country ด้วย id และกรองด้วยคำว่า WHERE country = 'Spain'
# แต่มันออกมาต่างกันนิดหน่อย เลยใช้วิธีนี้ให้เหมือนมากกว่า
pd.read_sql('''SELECT id, Team_api_id, Team_long_name, Team_short_name
               FROM Team
               WHERE id BETWEEN 43035 AND 47615
               AND id NOT IN (43041, 43048, 43800, 43804, 44557) 
               ORDER BY id LIMIT 50''', engine)

,id,team_api_id,team_long_name,team_short_name
0,43035,10267,Valencia CF,VAL
1,43036,8661,RCD Mallorca,MAL
2,43037,8371,CA Osasuna,OSA
3,43038,10205,Villarreal CF,VIL
4,43039,9783,RC Deportivo de La Coruña,COR
5,43040,8633,Real Madrid CF,REA
6,43042,8634,FC Barcelona,BAR
7,43043,8696,Racing Santander,SAN
8,43044,8302,Sevilla FC,SEV
9,43045,9869,Real Sporting de Gijón,SPG


In [21]:
# matches_spain
pd.read_sql_query('''SELECT id,	country_id,	season,	stage, date, home_team_api_id AS hometeam_id, away_team_api_id AS awayteam_id, home_team_goal AS home_goal, away_team_goal AS away_goal
                     FROM Match
                     WHERE id BETWEEN 22658 AND 24177''', engine)

,id,country_id,season,stage,date,hometeam_id,awayteam_id,home_goal,away_goal
0,22658,21518,2011/2012,1,2012-01-21 00:00:00,8560,9906,0,4
1,22659,21518,2011/2012,1,2012-01-22 00:00:00,8633,8315,4,1
2,22660,21518,2011/2012,1,2012-01-22 00:00:00,9864,8634,1,4
3,22661,21518,2011/2012,1,2012-01-23 00:00:00,10205,9869,3,0
4,22662,21518,2011/2012,1,2012-01-21 00:00:00,8603,8302,1,1
...,...,...,...,...,...,...,...,...,...
1515,24173,21518,2014/2015,9,2014-10-25 00:00:00,9865,8315,0,1
1516,24174,21518,2014/2015,9,2014-10-24 00:00:00,9910,8581,3,0
1517,24175,21518,2014/2015,9,2014-10-25 00:00:00,8633,8634,3,1
1518,24176,21518,2014/2015,9,2014-10-25 00:00:00,8372,7878,1,1


### **In CASE of rivalry**

Barcelona and Real Madrid have been rival teams for more than 80 years. Matches between these two teams are given the name *El Clásico* (The Classic). In this exercise, you will query a list of matches played between these two rivals.

You will notice in Step 2 that when you have multiple logical conditions in a `CASE` statement, you may quickly end up with a large number of `WHEN` clauses to logically test every outcome you are interested in. It's important to make sure you don't accidentally exclude key information in your `ELSE` clause.

In this exercise, you will retrieve information about matches played between **Barcelona** (id = `8634`) and **Real Madrid** (id = `8633`). Note that the query you are provided with already identifies the *Clásico* matches using a filter in the `WHERE` clause.

* Complete the first `CASE` statement, identifying Barcelona or Real Madrid as the home team using the `hometeam_id` column.
* Complete the second `CASE` statement in the same way, using `awayteam_id`.

In [22]:
pd.read_sql_query('''SELECT date, 
	-- Identify the home team as Barcelona or Real Madrid
                            CASE WHEN hometeam_id = 8634 THEN 'FC Barcelona' 
                                 ELSE 'Real Madrid CF' END AS home,
    -- Identify the away team as Barcelona or Real Madrid
                            CASE WHEN awayteam_id = 8634 THEN 'FC Barcelona' 
                                 ELSE 'Real Madrid CF' END AS away
    -- ตรงนี้ แทนที่ matche_spain ในแบบฝึกหัดด้วย subqueries
                     FROM (SELECT id,	country_id,	season,	stage, date, home_team_api_id AS hometeam_id, away_team_api_id AS awayteam_id, home_team_goal AS home_goal, away_team_goal AS away_goal
                           FROM Match
                           WHERE id BETWEEN 22658 AND 24177) AS matches_spain
                     WHERE (awayteam_id = 8634 OR hometeam_id = 8634)
                     AND (awayteam_id = 8633 OR hometeam_id = 8633);''', engine)

,date,home,away
0,2011-12-10 00:00:00,Real Madrid CF,FC Barcelona
1,2012-04-21 00:00:00,FC Barcelona,Real Madrid CF
2,2013-03-02 00:00:00,Real Madrid CF,FC Barcelona
3,2012-10-07 00:00:00,FC Barcelona,Real Madrid CF
4,2013-10-26 00:00:00,FC Barcelona,Real Madrid CF
5,2014-03-23 00:00:00,Real Madrid CF,FC Barcelona
6,2015-03-22 00:00:00,FC Barcelona,Real Madrid CF
7,2014-10-25 00:00:00,Real Madrid CF,FC Barcelona


* Construct the final `CASE` statement identifying who won each match. Note there are 3 possible outcomes, but 5 conditions that you need to identify.
* Fill in the logical operators to identify Barcelona or Real Madrid as the winner.

In [23]:
pd.read_sql_query('''SELECT date, 
                            CASE WHEN hometeam_id = 8634 THEN 'FC Barcelona' ELSE 'Real Madrid CF' END as home,
                            CASE WHEN awayteam_id = 8634 THEN 'FC Barcelona' ELSE 'Real Madrid CF' END as away,
	-- Identify all possible match outcomes
                            CASE WHEN home_goal  > away_goal AND hometeam_id = 8634 THEN 'Barcelona win!'
                            WHEN home_goal  > away_goal AND hometeam_id = 8633 THEN 'Real Madrid win!'
                            WHEN home_goal  < away_goal AND awayteam_id = 8634 THEN 'Barcelona win!'
                            WHEN home_goal  < away_goal AND awayteam_id = 8633 THEN 'Real Madrid win!'
                            ELSE 'Tie!' END AS outcome
                     FROM (SELECT id,	country_id,	season,	stage, date, home_team_api_id AS hometeam_id, away_team_api_id AS awayteam_id, home_team_goal AS home_goal, away_team_goal AS away_goal
                           FROM Match
                           WHERE id BETWEEN 22658 AND 24177) AS matches_spain
                     WHERE (awayteam_id = 8634 OR hometeam_id = 8634)
                     AND (awayteam_id = 8633 OR hometeam_id = 8633);''', engine)

,date,home,away,outcome
0,2011-12-10 00:00:00,Real Madrid CF,FC Barcelona,Barcelona win!
1,2012-04-21 00:00:00,FC Barcelona,Real Madrid CF,Real Madrid win!
2,2013-03-02 00:00:00,Real Madrid CF,FC Barcelona,Real Madrid win!
3,2012-10-07 00:00:00,FC Barcelona,Real Madrid CF,Tie!
4,2013-10-26 00:00:00,FC Barcelona,Real Madrid CF,Barcelona win!
5,2014-03-23 00:00:00,Real Madrid CF,FC Barcelona,Barcelona win!
6,2015-03-22 00:00:00,FC Barcelona,Real Madrid CF,Barcelona win!
7,2014-10-25 00:00:00,Real Madrid CF,FC Barcelona,Real Madrid win!


แบบฝึกหัดถัดไปใช้ข้อมูลของอิตาลี


In [24]:
# teams_italy
pd.read_sql('''SELECT id, Team_api_id, Team_long_name, Team_short_name
               FROM Team
               WHERE id BETWEEN 20513 AND 25048
               AND id NOT IN (20520, 21280, 22044) 
               ORDER BY id LIMIT 50''', engine)

,id,team_api_id,team_long_name,team_short_name
0,20513,8524,Atalanta,ATA
1,20514,8551,Siena,SIE
2,20515,8529,Cagliari,CAG
3,20516,8543,Lazio,LAZ
4,20517,8530,Catania,CAT
5,20518,10233,Genoa,GEN
6,20519,8533,Chievo Verona,CHI
7,20521,8535,Fiorentina,FIO
8,20522,9885,Juventus,JUV
9,20523,8564,Milan,ACM


In [25]:
# matches_italy
pd.read_sql_query('''SELECT id,	country_id,	season,	stage, date, home_team_api_id AS hometeam_id, away_team_api_id AS awayteam_id, home_team_goal AS home_goal, away_team_goal AS away_goal
                     FROM Match
                     WHERE id BETWEEN 11397 AND 12893''', engine)

,id,country_id,season,stage,date,hometeam_id,awayteam_id,home_goal,away_goal
0,11397,10257,2011/2012,1,2011-12-21 00:00:00,8524,9880,4,1
1,11398,10257,2011/2012,1,2011-12-21 00:00:00,9857,8686,0,2
2,11399,10257,2011/2012,1,2011-12-20 00:00:00,8529,8564,0,2
3,11400,10257,2011/2012,1,2011-12-21 00:00:00,8636,9888,4,1
4,11401,10257,2011/2012,1,2011-12-21 00:00:00,8543,8533,0,0
...,...,...,...,...,...,...,...,...,...
1492,12889,10257,2014/2015,9,2014-10-29 00:00:00,8636,9882,1,0
1493,12890,10257,2014/2015,9,2014-10-29 00:00:00,8540,8533,1,0
1494,12891,10257,2014/2015,9,2014-10-29 00:00:00,8686,9880,2,0
1495,12892,10257,2014/2015,9,2014-10-28 00:00:00,7943,8534,3,1


## **Filtering your `CASE` statement**

Let's generate a list of matches <ins>won</ins> by Italy's *Bologna* team! There are quite a few additional teams in the two tables, so a key part of generating a usable query will be using your `CASE` statement as a filter in the `WHERE` clause.

`CASE` statements allow you to categorize data that you're interested in -- and exclude data you're not interested in. In order to do this, you can use a `CASE` statement as a filter in the `WHERE` statement to remove output you don't want to see.

Here is how you might set that up:

```
SELECT *
FROM table
WHERE 
    CASE WHEN a > 5 THEN 'Keep'
         WHEN a <= 5 THEN 'Exclude' END = 'Keep';
```         
         
         
In essence, you can use the `CASE` statement as a filtering column like any other column in your database. The only difference is that <ins>you don't alias the statement in `WHERE`</ins>.

* Identify `Bologna`'s team ID listed in the `teams_italy` table by selecting the `team_long_name` and `team_api_id`.

In [26]:
pd.read_sql_query('''SELECT team_long_name, team_api_id
                     FROM (SELECT id, Team_api_id, Team_long_name, Team_short_name
                           FROM Team
                           WHERE id BETWEEN 20513 AND 25048
                           AND id NOT IN (20520, 21280, 22044) 
                           ORDER BY id) AS teams_italy
                     WHERE team_long_name = 'Bologna';''', engine)

,Team_long_name,Team_api_id
0,Bologna,9857


* Select the `season` and `date` that a match was played.
* Complete the `CASE` statement so that only Bologna's home and away wins are identified.

In [27]:
pd.read_sql_query('''SELECT season, date, CASE WHEN hometeam_id = 9857 AND home_goal > away_goal THEN 'Bologna Win'
                                               WHEN awayteam_id = 9857 AND away_goal > home_goal THEN 'Bologna Win' 
                                               END AS outcome
                      FROM (SELECT id,	country_id,	season,	stage, date, home_team_api_id AS hometeam_id, away_team_api_id AS awayteam_id, home_team_goal AS home_goal, away_team_goal AS away_goal
                            FROM Match
                            WHERE id BETWEEN 11397 AND 12893) AS matches_italy;''', engine)

,season,date,outcome
0,2011/2012,2011-12-21 00:00:00,None
1,2011/2012,2011-12-21 00:00:00,None
2,2011/2012,2011-12-20 00:00:00,None
3,2011/2012,2011-12-21 00:00:00,None
4,2011/2012,2011-12-21 00:00:00,None
...,...,...,...
1492,2014/2015,2014-10-29 00:00:00,None
1493,2014/2015,2014-10-29 00:00:00,None
1494,2014/2015,2014-10-29 00:00:00,None
1495,2014/2015,2014-10-28 00:00:00,None


* Select the `home_goal` and `away_goal` for each match.
* Use the `CASE` statement in the `WHERE` clause to filter all `NULL` values generated by the statement in the previous step.

In [28]:
pd.read_sql_query('''
SELECT season, date, home_goal,	away_goal,
       CASE WHEN hometeam_id = 9857 AND home_goal > away_goal THEN 'Bologna Home Win'
		    WHEN awayteam_id = 9857 AND away_goal > home_goal THEN 'Bologna Away Win' 
            END AS outcome

FROM (SELECT id,	country_id,	season,	stage, date, home_team_api_id AS hometeam_id, away_team_api_id AS awayteam_id, home_team_goal AS home_goal, away_team_goal AS away_goal
                            FROM Match
                            WHERE id BETWEEN 11397 AND 12893) AS matches_italy
WHERE 
-- Exclude games not won by Bologna
	CASE WHEN hometeam_id = 9857 AND home_goal > away_goal THEN 'Bologna Win'
		 WHEN awayteam_id = 9857 AND away_goal > home_goal THEN 'Bologna Win' 
		 END IS NOT NULL;''', engine)

,season,date,home_goal,away_goal,outcome
0,2011/2012,2011-10-30 00:00:00,3,1,Bologna Home Win
1,2011/2012,2011-12-04 00:00:00,1,0,Bologna Home Win
2,2011/2012,2012-01-08 00:00:00,2,0,Bologna Home Win
3,2011/2012,2012-02-21 00:00:00,2,0,Bologna Home Win
4,2011/2012,2012-02-17 00:00:00,0,3,Bologna Away Win
5,2011/2012,2012-04-12 00:00:00,1,0,Bologna Home Win
6,2011/2012,2012-04-29 00:00:00,3,2,Bologna Home Win
7,2011/2012,2012-05-02 00:00:00,0,1,Bologna Away Win
8,2011/2012,2012-05-06 00:00:00,2,0,Bologna Home Win
9,2011/2012,2011-10-16 00:00:00,0,2,Bologna Away Win


## **Counting CASES**

* Count the number of home and away games that Liverpool won in each season
* `CASE` สามารถนำไปใส่ใน aggregate functions ได้

## **CASE WHEN with COUNT**

* จุดที่แตกต่างคือสิ่งที่เขียนตามหลัง `THEN` แทนที่จะเขียนเป็นตัวหนังสือ ให้ใช้คอลัมน์ที่ไม่มีค่าซ้ำ เช่น `id`
    * จริงๆแล้วจะเป็นค่าอะไรก็ได้
* พอ `CASE` นี้ไปอยู่ใน `COUNT` ก็เลยเป็นการนับค่าตามเงื่อนไขที่เกิดขึ้นนั่นเอง

In [29]:
pd.read_sql_query('''
SELECT season,
       COUNT(CASE WHEN home_team_api_id = 8650 AND home_team_goal > away_team_goal THEN id END) AS home_wins,
       COUNT(CASE WHEN away_team_api_id = 8650 AND home_team_goal < away_team_goal THEN id END) AS away_wins
FROM Match
GROUP BY season;''', engine)

,season,home_wins,away_wins
0,2008/2009,12,13
1,2009/2010,13,5
2,2010/2011,12,5
3,2011/2012,6,8
4,2012/2013,9,7
5,2013/2014,16,10
6,2014/2015,10,8
7,2015/2016,8,8


# **CASE WHEN with SUM**

* สมมติสนใจจำนวนประตูที่ลิเวอร์พูลทำแต้มได้ในแต่ละฤดูกาล

In [30]:
pd.read_sql_query('''
SELECT season,
       SUM(CASE WHEN home_team_api_id = 8650 THEN home_team_goal END) AS home_goals,
       SUM(CASE WHEN away_team_api_id = 8650 THEN away_team_goal END) AS away_goals
FROM Match
GROUP BY season;''', engine)

,season,home_goals,away_goals
0,2008/2009,41,36
1,2009/2010,43,18
2,2010/2011,37,22
3,2011/2012,24,23
4,2012/2013,33,38
5,2013/2014,53,48
6,2014/2015,30,22
7,2015/2016,33,30


# **CASE with AVG (1)**

In [31]:
pd.read_sql_query('''
SELECT season,
       ROUND(AVG(CASE WHEN home_team_api_id = 8650 THEN home_team_goal END),2) AS home_goals,
       ROUND(AVG(CASE WHEN away_team_api_id = 8650 THEN away_team_goal END),2) AS away_goals
FROM Match
GROUP BY season;''', engine)

,season,home_goals,away_goals
0,2008/2009,2.16,1.89
1,2009/2010,2.26,0.95
2,2010/2011,1.95,1.16
3,2011/2012,1.26,1.21
4,2012/2013,1.74,2.00
5,2013/2014,2.79,2.53
6,2014/2015,1.58,1.16
7,2015/2016,1.74,1.58


# **Percentages with CASE and AVG**

* หาร้อยละที่เชลซีชนะ

In [32]:
pd.read_sql_query('''
SELECT season, team_long_name,
       ROUND(AVG(CASE WHEN home_team_api_id = 8455 AND home_team_goal > away_team_goal THEN 1 
                      WHEN home_team_api_id = 8455 AND home_team_goal < away_team_goal THEN 0
                      END), 2) AS pct_homewins,
       ROUND(AVG(CASE WHEN away_team_api_id = 8455 AND home_team_goal < away_team_goal THEN 1 
                      WHEN away_team_api_id = 8455 AND home_team_goal > away_team_goal THEN 0
                      END), 2) AS pct_awaywins
FROM Match
INNER JOIN Team ON (Team.team_api_id = Match.home_team_api_id) OR (Team.team_api_id = Match.away_team_api_id)
WHERE Team.team_api_id = 8455
GROUP BY season;''', engine)

,season,team_long_name,pct_homewins,pct_awaywins
0,2008/2009,Chelsea,0.85,0.82
1,2009/2010,Chelsea,0.94,0.67
2,2010/2011,Chelsea,0.88,0.50
3,2011/2012,Chelsea,0.75,0.50
4,2012/2013,Chelsea,0.86,0.67
5,2013/2014,Chelsea,0.94,0.67
6,2014/2015,Chelsea,1.00,0.79
7,2015/2016,Chelsea,0.50,0.50


### **COUNT using CASE WHEN**

Do the number of soccer matches played in a given European country differ across seasons? 

You will examine the number of matches played in 3 seasons within each country listed in the database. This is much easier to explore with each season's matches in separate columns. Using the `Country` and unfiltered `Match` table, you will count the number of matches played in each country during the 2012/2013, 2013/2014, and 2014/2015 match seasons.

* Create a `CASE` statement that identifies the `id` of matches played in the 2012/2013 season. Specify that you want `ELSE` values to be `NULL`.
* Wrap the `CASE` statement in a `COUNT` function and group the query by the `country` alias.

In [33]:
pd.read_sql_query('''
SELECT c.name AS country,
    -- Count games from the 2012/2013 season
       COUNT(CASE WHEN m.season = '2012/2013' THEN 'whatever' ELSE NULL END) AS matches_2012_2013
FROM Country c
LEFT JOIN Match m
ON c.id = m.country_id
-- Group by country name alias
GROUP BY c.name;''', engine)

,country,matches_2012_2013
0,Belgium,240
1,England,380
2,France,380
3,Germany,306
4,Italy,380
5,Netherlands,306
6,Poland,240
7,Portugal,240
8,Scotland,228
9,Spain,380


* Create 3 `CASE WHEN` statements counting the matches played in each country across the 3 seasons.
* `END` your `CASE` statement without an `ELSE` clause.

In [34]:
pd.read_sql_query('''
SELECT c.name AS country,
       COUNT(CASE WHEN m.season = '2012/2013' THEN m.id END) AS matches_2012_2013,
	   COUNT(CASE WHEN m.season = '2013/2014' THEN m.id END) AS matches_2013_2014,
	   COUNT(CASE WHEN m.season = '2014/2015' THEN m.id END) AS matches_2014_2015
FROM country c
LEFT JOIN match m
ON c.id = m.country_id
-- Group by country name alias
GROUP BY country;''', engine)

,country,matches_2012_2013,matches_2013_2014,matches_2014_2015
0,Belgium,240,12,240
1,England,380,380,380
2,France,380,380,380
3,Germany,306,306,306
4,Italy,380,380,379
5,Netherlands,306,306,306
6,Poland,240,240,240
7,Portugal,240,240,306
8,Scotland,228,228,228
9,Spain,380,380,380


### **COUNT and CASE WHEN with multiple conditions**

In Python, you have the ability to calculate a `SUM` of logical values (i.e., `TRUE`/`FALSE`) directly. In SQL, you have to convert these values into 1 and 0 before calculating a sum. This can be done using a `CASE` statement.

There's one key difference when using `SUM` to aggregate logical values compared to using `COUNT` in the previous exercise.

Your goal here is to use the `Country` and `Match` table to determine the *total number of matches won by the home team in each country* during the 2012/2013, 2013/2014, and 2014/2015 seasons.

* Create 3 `CASE` statements to "count" matches in the `'2012/2013'`, `'2013/2014'`, and `'2014/2015'` seasons, respectively.
* Have each `CASE` statement return a `1` for every match you want to include, and a `0` for every match to exclude.
* Wrap the `CASE` statement in a `SUM` to return the total matches played in each season.
* Group the query by the country name alias.

In [35]:
pd.read_sql_query('''
SELECT name AS country,
	   SUM(CASE WHEN season = '2012/2013' AND home_team_goal > away_team_goal THEN 1 ELSE 0 END) AS matches_2012_2013,
 	   SUM(CASE WHEN season = '2013/2014' AND home_team_goal > away_team_goal THEN 1 ELSE 0 END) AS matches_2013_2014,
       SUM(CASE WHEN season = '2014/2015' AND home_team_goal > away_team_goal THEN 1 ELSE 0 END) AS matches_2014_2015
FROM Country
LEFT JOIN Match
ON Country.id = Match.country_id
GROUP BY country;''', engine)

,country,matches_2012_2013,matches_2013_2014,matches_2014_2015
0,Belgium,102,6,106
1,England,166,179,172
2,France,170,168,181
3,Germany,130,145,145
4,Italy,177,181,152
5,Netherlands,137,144,138
6,Poland,97,110,114
7,Portugal,103,108,137
8,Scotland,89,102,102
9,Spain,189,179,171


### **Calculating percent with CASE and AVG**

`CASE` statements will return any value you specify in your `THEN` clause. This is an incredibly powerful tool for robust calculations and data manipulation when used in conjunction with an aggregate statement. One key task you can perform is using `CASE` inside an `AVG` function to calculate a percentage of information in your database.

Here's an example of how you set that up:

```
AVG(CASE WHEN condition_is_met THEN 1
         WHEN condition_is_not_met THEN 0 END)
```

With this approach, it's important to accurately specify which records count as `0`, otherwise your calculations may not be correct!



Your task is to examine the number of wins, losses, and ties in each country. The `Match`es table is filtered to include all matches from the 2013/2014 and 2014/2015 seasons.

* Create 3 `CASE` statements to `COUNT` the total number of home team wins, away team wins, and ties, which will allow you to examine the total number of records.

In [36]:
pd.read_sql_query('''
SELECT name AS country, 
COUNT(CASE WHEN home_team_goal > away_team_goal AND (season = '2013/2014' OR season = '2014/2015') THEN 'You can type' END) AS home_wins,
COUNT(CASE WHEN home_team_goal < away_team_goal AND (season = '2013/2014' OR season = '2014/2015') THEN 'whatever for' END) AS away_wins,
COUNT(CASE WHEN home_team_goal = away_team_goal AND (season = '2013/2014' OR season = '2014/2015') THEN 'COUNT CASE'   END) AS ties
FROM Country
LEFT JOIN Match
ON Country.id = Match.country_id
GROUP BY country;''', engine)

,country,home_wins,away_wins,ties
0,Belgium,112,78,62
1,England,351,238,171
2,France,349,215,196
3,Germany,290,176,146
4,Italy,333,216,210
5,Netherlands,282,173,157
6,Poland,224,117,139
7,Portugal,245,156,145
8,Scotland,204,158,94
9,Spain,350,233,177


* Calculate the percentage of matches tied using a `CASE` statement inside `AVG`.
* Fill in the logical operators for each statement. Alias your columns as `ties_2013_2014` and `ties_2014_2015`, respectively.

In [37]:
pd.read_sql_query('''
SELECT name AS country,
AVG(CASE WHEN season='2013/2014' AND home_team_goal =  away_team_goal THEN 1 
		 WHEN season='2013/2014' AND home_team_goal <>  away_team_goal THEN 0 END) AS ties_2013_2014,
AVG(CASE WHEN season='2014/2015' AND home_team_goal =  away_team_goal  THEN 1 
		 WHEN season='2014/2015' AND home_team_goal <>  away_team_goal THEN 0 END) AS ties_2014_2015
FROM Country
LEFT JOIN Match
ON Country.id = Match.country_id
GROUP BY country;''', engine)

,country,ties_2013_2014,ties_2014_2015
0,Belgium,0.166667,0.250000
1,England,0.205263,0.244737
2,France,0.284211,0.231579
3,Germany,0.209150,0.267974
4,Italy,0.236842,0.316623
5,Netherlands,0.274510,0.238562
6,Poland,0.304167,0.275000
7,Portugal,0.250000,0.277778
8,Scotland,0.219298,0.192982
9,Spain,0.226316,0.239474


* The previous "ties" columns returned values with 14 decimal points, which is not easy to interpret. Use the `ROUND` function to round to 2 decimal points.

In [38]:
pd.read_sql_query('''
SELECT name AS country,
ROUND(AVG(CASE WHEN season='2013/2014' AND home_team_goal =  away_team_goal THEN 1 
		 WHEN season='2013/2014' AND home_team_goal <>  away_team_goal THEN 0 END), 2) AS ties_2013_2014,
ROUND(AVG(CASE WHEN season='2014/2015' AND home_team_goal =  away_team_goal  THEN 1 
		 WHEN season='2014/2015' AND home_team_goal <>  away_team_goal THEN 0 END), 2) AS ties_2014_2015
FROM Country
LEFT JOIN Match
ON Country.id = Match.country_id
GROUP BY country;''', engine)

,country,ties_2013_2014,ties_2014_2015
0,Belgium,0.17,0.25
1,England,0.21,0.24
2,France,0.28,0.23
3,Germany,0.21,0.27
4,Italy,0.24,0.32
5,Netherlands,0.27,0.24
6,Poland,0.30,0.28
7,Portugal,0.25,0.28
8,Scotland,0.22,0.19
9,Spain,0.23,0.24


# **Simple Subqueries**

* Can be queried independently from the outer query
* Executed first before the main query

In [39]:
pd.read_sql_query(''' 
SELECT home_team_goal
FROM Match
WHERE home_team_goal > 
      (SELECT AVG(home_team_goal) FROM Match)
LIMIT 5;''', engine)

,home_team_goal
0,5
1,2
2,4
3,2
4,2


## **Subqueries in WHERE**

* สมมติอยากทราบว่า ในฤดูกาล 2012/2013 มีการแข่งขันกี่คู่ที่มี `home_team_goal > away_team_goal`

In [40]:
pd.read_sql_query('''
SELECT date, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal
FROM Match
WHERE season = '2012/2013'
AND home_team_goal > (SELECT AVG(home_team_goal) FROM Match)
LIMIT 10; ''', engine)

,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,2012-07-28 00:00:00,9993,9994,2,4
1,2012-07-28 00:00:00,9998,1773,5,2
2,2012-07-28 00:00:00,8203,9986,4,2
3,2012-07-28 00:00:00,8342,8475,3,1
4,2012-07-29 00:00:00,9991,9989,2,0
5,2012-07-29 00:00:00,9987,9984,3,3
6,2012-10-07 00:00:00,9985,8635,2,1
7,2012-10-05 00:00:00,9993,9991,2,2
8,2012-10-06 00:00:00,10000,9984,3,1
9,2012-10-06 00:00:00,9994,9989,2,2


## **Subqueries filtering list with IN**

* ทีมไหนมาจากประเทศโปแลนด์?

In [41]:
display(pd.read_sql_query("SELECT * FROM Country WHERE name = 'Poland'", engine))
pd.read_sql_query('''SELECT team_long_name, team_short_name AS abbr
                     FROM Team 
                     WHERE team_api_id IN (SELECT home_team_api_id FROM Match WHERE country_id = 15722)
                     ORDER BY abbr;''', engine)

,id,name
0,15722,Poland


,team_long_name,abbr
0,Arka Gdynia,ARK
1,GKS Bełchatów,BEL
2,Jagiellonia Białystok,BIA
3,Ruch Chorzów,CHO
4,Cracovia,CKR
5,Polonia Bytom,GOR
6,Korona Kielce,KKI
7,Górnik Łęczna,LEC
8,Legia Warszawa,LEG
9,Lechia Gdańsk,LGD


In [42]:
# เขียนด้วย INNER JOIN ก็พอไหวอยู่
pd.read_sql_query(''' 
SELECT DISTINCT team_long_name, team_short_name AS abbr
FROM Team
INNER JOIN Match ON (Team.team_api_id = Match.home_team_api_id) OR (Team.team_api_id = Match.away_team_api_id)
INNER JOIN Country ON Country.id = Match.country_id
WHERE name = 'Poland'
ORDER BY abbr''', engine)

,team_long_name,abbr
0,Arka Gdynia,ARK
1,GKS Bełchatów,BEL
2,Jagiellonia Białystok,BIA
3,Ruch Chorzów,CHO
4,Cracovia,CKR
5,Polonia Bytom,GOR
6,Korona Kielce,KKI
7,Górnik Łęczna,LEC
8,Legia Warszawa,LEG
9,Lechia Gdańsk,LGD


### **Filtering using scalar subqueries**

Subqueries are incredibly powerful for performing complex filters and transformations. You can filter data based on single, *scalar* values using a subquery in ways you cannot by using `WHERE` statements or joins. Subqueries can also be used for more advanced manipulation of your data set. You will likely encounter subqueries in any real-world setting that uses relational databases.

In this exercise, you will generate a list of matches where the total goals scored (for both teams in total) is more than 3 times the average for games in the `Match` table, which includes only games played in the 2013/2014 season.

* Calculate *triple *the average home + away goals scored across all matches. This will become your subquery in the next step. Note that this column does not have an alias, so it will be called `?column?` in your results.

In [43]:
pd.read_sql('''SELECT 3 * AVG(home_team_goal + away_team_goal) 
               FROM Match 
               WHERE season = '2013/2014';''', engine)

,3 * AVG(home_team_goal + away_team_goal)
0,8.300462


* Select the `date`, `home_team_goal`, and `away_team_goal` in the main query.
* Filter the main query for matches where the total goals scored exceed the value in the subquery.

In [44]:
pd.read_sql(''' 
SELECT date, home_team_goal, away_team_goal
FROM Match
WHERE season = '2013/2014'
AND home_team_goal + away_team_goal > (SELECT 3 * AVG(home_team_goal + away_team_goal) 
                                       FROM Match 
                                       WHERE season = '2013/2014');''', engine)

,date,home_team_goal,away_team_goal
0,2013-12-14 00:00:00,6,3
1,2014-03-22 00:00:00,3,6
2,2013-10-30 00:00:00,7,3


### **Filtering using a subquery with a list**

Your goal in this exercise is to generate a list of teams that never played a game in their home city. Using a subquery, you will generate a list of unique `home_team_api_id` values from the unfiltered `?atch` table to exclude in the team table's `team_api_id` column.

In addition to filtering using a single-value (scalar) subquery, you can create a list of values in a subquery to filter data based on a complex set of conditions. This type of subquery generates <ins>a one column reference list</ins> for the main query. As long as the values in your list match a column in your main query's table, you don't need to use a join -- even if the list is from a separate table.

* Create a subquery in the `WHERE` clause that retrieves all unique `home_team_api_id` values from the `Match` table.
* Select the `team_long_name` and `team_short_name` from the `Team` table. Exclude all values from the subquery in the main query.

In [45]:
pd.read_sql_query('''
SELECT team_long_name,
       team_short_name
FROM Team 
WHERE team_api_id NOT IN (1601,1773,1957,2033,2182,2183,2186,4064,4087,6269,6403,6413,6433,6547,6631,7730,7788,7794,7819,7841,7842,7844,7869,7878,7943,7955,8019,8020,8021,8023,8024,8025,8027,8028,8030,8033,8066,8165,8177,8178,8191,8194,8197,8203,8226,8244,8245,8277,8284,8302,8305,8315,8342,8344,8348,8350,8357,8358,8370,8371,8372,8394,8406,8426,8429,8455,8456,8457,8460,8464,8466,8467,8472,8475,8481,8485,8521,8524,8525,8526,8528,8529,8530,8533,8534,8535,8537,8540,8543,8548,8550,8551,8558,8559,8560,8564,8569,8571,8573,8576,8581,8583,8586,8588,8592,8593,8597,8600,8602,8603,8611,8613,8633,8634,8635,8636,8639,8640,8649,8650,8654,8655,8659,8661,8667,8668,8673,8674,8686,8689,8696,8697,8721,8722,9747,9748,9761,9764,9768,9771,9772,9773,9776,9777,9783,9788,9789,9790,9791,9798,9800,9803,9804,9807,9810,9823,9824,9825,9826,9827,9829,9830,9831,9836,9837,9839,9847,9850,9851,9853,9857,9860,9864,9865,9869,9873,9874,9875,9876,9878,9879,9880,9882,9885,9888,9904,9905,9906,9908,9910,9925,9927,9930,9931,9938,9941,9956,9984,9985,9986,9987,9989,9991,9993,9994,9997,9998,10000,10001,10003,10167,10172,10179,10189,10190,10191,10192,10194,10199,10205,10211,10212,10214,10215,10217,10218,10219,10228,10229,10233,10235,10238,10242,10243,10249,10251,10252,10260,10261,10264,10265,10267,10268,10269,10281,158085,274581
);''', engine)

,team_long_name,team_short_name
0,FCV Dender EH,DEN
1,KSV Roeselare,ROS
2,Tubize,TUB
3,Royal Excel Mouscron,MOU
4,KAS Eupen,EUP
5,Middlesbrough,MID
6,Portsmouth,POR
7,Birmingham City,BIR
8,Blackpool,BLA
9,Bournemouth,BOU


In [46]:
# ข้อมูลจริง ทำไมได้แบบนี้ สงสัยเป็นเพราะตาราง Match บน Kaggle กับ matches บน datacamp มีจำนวนแถวที่ต่างกันเยอะมาก
pd.read_sql_query('''
SELECT team_long_name,
       team_short_name
FROM Team 
WHERE team_api_id NOT IN (SELECT DISTINCT home_team_api_id FROM match);''', engine)

,team_long_name,team_short_name


### **Filtering with more complex subquery conditions**
In the previous exercise, you generated a list of teams that have no home matches listed in the soccer database using a subquery in `WHERE`. Let's do some further exploration in this database by creating a list of teams that scored *8 or more goals in a home match*.

In order to do this, you will construct a subquery in the `WHERE` statement with its own filtering condition.

* Create a subquery in `WHERE` clause that retrieves all `home_team_api_id` values from `Match` with a `home_team_goal` score greater than or equal to `8`.
* Select the `team_long_name` and `team_short_name` from the `Team` table. Include all values from the subquery in the main query.

### * `season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')` ถูกใส่มาเพื่อให้ข้อมูล Match ใน Kaggle ตรงกับบน Datacamp

In [47]:
pd.read_sql_query('''
SELECT 	team_long_name,team_short_name
FROM Team
WHERE team_api_id IN (SELECT home_team_api_id
       				  FROM Match
       				  WHERE home_team_goal >= 8
                      AND season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'));''', engine)

,team_long_name,team_short_name
0,Chelsea,CHE
1,Southampton,SOU
2,Real Madrid CF,REA
3,FC Barcelona,BAR
4,FC Bayern Munich,BMU
5,Manchester United,MUN


# **Subqueries in FROM**

* เหมือนกับการเลือกข้อมูลจากตารางใหม่ แต่ตารางนั้นถูกสร้างมาภายใต้คำสั่งที่เขียนใน subquery
* ตัวอย่างเช่น สมมติต้องการหาทีมี่ได้ highest average home goals.

## **FROM Subqueries**

In [48]:
pd.read_sql_query('''
SELECT team_long_name AS team, AVG(home_team_goal) AS home_avg
FROM Match
LEFT JOIN Team
ON Match.home_team_api_id = Team.team_api_id
WHERE season = '2011/2012'
GROUP BY Team
LIMIT 5;''', engine)

,team,home_avg
0,1. FC Kaiserslautern,0.705882
1,1. FC Köln,1.176471
2,1. FC Nürnberg,1.294118
3,1. FSV Mainz 05,1.588235
4,AC Ajaccio,1.157895


ตัวอย่างไม่ค่อยดี เขียนปกติ ไม่ต้องทำเป็น subquery ก็ได้

In [49]:
pd.read_sql_query('''
SELECT team_long_name AS team, AVG(home_team_goal) AS home_avg
FROM Match
LEFT JOIN Team
ON Match.home_team_api_id = Team.team_api_id
WHERE season = '2011/2012'
GROUP BY Team
ORDER BY home_avg DESC
LIMIT 5;''', engine)

,team,home_avg
0,FC Barcelona,3.842105
1,Real Madrid CF,3.684211
2,PSV,3.352941
3,SL Benfica,2.933333
4,RSC Anderlecht,2.933333


ถ้าจะเขียนเป็น subquery ก็ต้องระวัง
* ใส่ subquery ในวงเล็บ
* เอาเครื่องหมาย ; ออกจาก subquery
* จะใส่ alias หรือไม่ใส่ให้ subquery ก็ได้เหมือนกัน


In [50]:
pd.read_sql_query('''
SELECT team, home_avg
FROM (SELECT team_long_name AS team, AVG(home_team_goal) AS home_avg
      FROM Match
      LEFT JOIN Team
      ON Match.home_team_api_id = Team.team_api_id
      WHERE season = '2011/2012'
      GROUP BY Team)
ORDER BY home_avg DESC
LIMIT 3;''', engine)

,team,home_avg
0,FC Barcelona,3.842105
1,Real Madrid CF,3.684211
2,PSV,3.352941


## **Things to remember using subqueries in FROM**

* You can create multiple subqueries in one `FROM` statement -- กรณีนี้ ต้องใช้ Alias
* อาจจะต้อง join

### **Joining Subqueries in FROM**

The `Match` table in the European Soccer Database does not contain country or team names. You can get this information by joining it to the `Country` table, and use this to aggregate information, such as the number of matches played in each country.

If you're interested in filtering data from one of these tables, you can also create a subquery from one of the tables, and then join it to an existing table in the database. A subquery in `FROM` is an effective way of answering detailed questions that requires filtering or transforming data before including it in your final results.

Your goal in this exercise is to generate a subquery using the `Match` table, and then join that subquery to the `Country` table to calculate information about matches with 10 or more goals in total!

* Create the subquery to be used in the next step, which selects the `country_id` and match ID (`id`) from the `Match` table.
* Filter the query for matches with greater than or equal to `10` goals.

In [51]:
pd.read_sql_query('''
SELECT 
	-- Select the country ID and match ID
	country_id, 
    id 
FROM Match
-- Filter for matches with 10 or more goals in total
WHERE (home_team_goal + away_team_goal) >= 10
AND season IN ('2013/2014', '2012/2013','2014/2015','2011/2012');''', engine)

,country_id,id
0,1729,3093
1,1729,3369
2,1729,3566
3,7809,9211
4,13274,14224
5,21518,23444
6,21518,24016
7,21518,24114
8,21518,24123


* Construct a subquery that selects only matches with `10` or more total goals.
* `INNER JOIN` the subquery onto `Country` in the main query.
* Select `country_name` from `Country` and count the `id` column from `Match`.

In [52]:
pd.read_sql_query('''
SELECT name AS country_name, COUNT(sub.id) AS matches
FROM Country
INNER JOIN (SELECT country_id, id 
            FROM Match
            WHERE (home_team_goal+away_team_goal) >= 10
            AND season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) AS sub
ON Country.id = sub.country_id
GROUP BY name;''', engine)

,country_name,matches
0,England,3
1,Germany,1
2,Netherlands,1
3,Spain,4


จริงๆ แค่ `INNER JOIN` เฉยๆ ก็ได้ ไม่ต้องเขียนให้เป็น subquery

In [53]:
pd.read_sql_query('''
SELECT name AS country_name, COUNT(Match.id) AS matches
FROM Country
INNER JOIN Match
ON Country.id = Match.country_id
WHERE home_team_goal+away_team_goal >= 10
AND season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')
GROUP BY name;''', engine)

,country_name,matches
0,England,3
1,Germany,1
2,Netherlands,1
3,Spain,4


### **Building on Subqueries in FROM**

In the previous exercise, you found that England, Netherlands, Germany and Spain were the only countries that had matches in the database where 10 or more goals were scored overall. Let's find out some more details about those matches -- when they were played, during which seasons, and how many of the goals were home versus away goals.

You'll notice that in this exercise, the table <ins>alias is excluded for every column selected in the main query</ins>. This is because the *main query* is extracting data from the *subquery*, which is treated as a single table.

* Complete the subquery inside the `FROM` clause. Select the country `name` from the `Country` table, along with the `date`, `the home_team_goal`, the `away_team_goal`, and the total goals columns from the `Match` table.
* Create a column in the subquery that adds home and away goals, called `total_goals`. This will be used to filter the main query.
* Select the country, date, home goals, and away goals in the main query.
* Filter the main query for games with 10 or more total goals.

In [54]:
pd.read_sql_query('''
SELECT country, date, home_team_goal, away_team_goal
FROM (SELECT c.name AS country, 
     	     m.date, 
     		 m.home_team_goal, 
     		 m.away_team_goal,
             m.home_team_goal + m.away_team_goal AS total_goals
      FROM Match m
      LEFT JOIN Country c
      ON m.country_id = c.id
      AND season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) AS subq
WHERE total_goals >= 10
AND country IS NOT NULL;''', engine)

,country,date,home_team_goal,away_team_goal
0,England,2011-08-28 00:00:00,8,2
1,England,2012-12-29 00:00:00,7,3
2,England,2013-05-19 00:00:00,5,5
3,Germany,2013-03-30 00:00:00,9,2
4,Netherlands,2011-11-06 00:00:00,6,4
5,Spain,2013-10-30 00:00:00,7,3
6,Spain,2015-04-05 00:00:00,9,1
7,Spain,2015-05-23 00:00:00,7,3
8,Spain,2014-09-20 00:00:00,2,8


# **Subqueries in SELECT**

* เพราะว่า เราไม่สามารถดึงค่า aggregated values ของ ungrouped query ได้ จึงต้องใช้ subquery in `SELECT`
* Returns a **<ins>single AGGREGATED value</ins>**
    * Include aggregate values to compare individuals values
    
สมมติอยากทราบว่า มีการแข่งขันในแต่ละฤดูกาลเป็นเท่าไร เมื่อเปรียบเทียบกับจำนวนการแข่งขันทั้งหมด
   * อย่าลืมว่า ข้อมูลไม่เท่ากันกับบนเว็บดาต้าแคมพ์

In [55]:
display(pd.read_sql("SELECT COUNT(id) FROM Match", engine))
print("\n")
display(pd.read_sql("SELECT COUNT(id) FROM Match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')", engine))

,COUNT(id)
0,25979


,COUNT(id)
0,12837


In [56]:
pd.read_sql_query(''' 
SELECT season, COUNT(id) AS matches, 12837 AS total_matches
FROM Match
WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')
GROUP BY season''', engine)

,season,matches,total_matches
0,2011/2012,3220,12837
1,2012/2013,3260,12837
2,2013/2014,3032,12837
3,2014/2015,3325,12837


ถ้าใช้ subquery

In [57]:
pd.read_sql_query(''' 
SELECT season, COUNT(id) AS matches, 
    (SELECT COUNT(id) FROM Match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) AS total_matches
FROM Match
WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')
GROUP BY season''', engine)

,season,matches,total_matches
0,2011/2012,3220,12837
1,2012/2013,3260,12837
2,2013/2014,3032,12837
3,2014/2015,3325,12837


## **SELECT subquries for mathematical operations**

In [58]:
pd.read_sql_query('''SELECT ROUND(AVG(home_team_goal + away_team_goal), 2) FROM Match WHERE season = '2011/2012'; ''', engine)

,"ROUND(AVG(home_team_goal + away_team_goal), 2)"
0,2.72


สมมติ ต้องการคำนวณว่าในการแข่งขันแต่ละครั้งนั้น จำนวนประตูที่ทำได้มีความแตกต่างจากจำนวนประตูเฉลี่ยในฤดูกาล 2011/2012 อย่างไร
* คือจะคำนวณตัวเลข `2.72` ไว้ก่อนก็ได้ หรือจะใช้เป็น subquery ตามด้านล่าง

In [59]:
pd.read_sql_query(''' 
SELECT date,
       home_team_goal + away_team_goal AS goals,
       home_team_goal + away_team_goal - (SELECT AVG(home_team_goal + away_team_goal) 
                                          FROM Match 
                                          WHERE season = '2011/2012') AS diff
FROM Match
WHERE season = '2011/2012'
LIMIT 4; ''', engine)

,date,goals,diff
0,2011-07-29 00:00:00,3,0.28354
1,2011-07-30 00:00:00,2,-0.71646
2,2011-07-30 00:00:00,4,1.28354
3,2011-07-30 00:00:00,1,-1.71646


## **จงจำไว้ว่า**

* subqueries ใน `SELECT` statement จะต้องให้ค่าข้อมูลแค่ค่าเดียว ต่อ 1 แถว  <ins>และเป็นค่า aggregated value</ins>
* เพราะข้อมูลใน subquery จะเอาไปใช้ต่อกับทุกๆแถว
* ### เงื่อนไขใน `WHERE` statement จะต้องเหมือนกันทั้งใน subquery และ main query

### **Add a subquery to the SELECT clause**

Subqueries in `SELECT` statements generate a single value that allow you to pass an aggregate value down a data frame. This is useful for performing calculations on data within your database.

In the following exercise, you will construct a query that calculates the *average number of goals per match in each country's league*.

* In the subquery, select the average total goals by adding `home_team_goal` and `away_team_goal`.
* Filter the results so that only the average of goals in the `2013/2014` season is calculated.
* In the main query, select the average total goals by adding `home_team_goal` and `away_team_goal`. This calculates the average goals for each league.
* Filter the results in the main query the same way you filtered the subquery. Group the query by the league name.

In [60]:
pd.read_sql_query('''
SELECT name AS league,
    -- Select and round the league's total goals
       ROUND(AVG(home_team_goal + away_team_goal), 2) AS avg_goals,
    -- Select & round the average total goals for the season
       (SELECT ROUND(AVG(home_team_goal + away_team_goal), 2) FROM match WHERE season = '2013/2014') AS overall_avg
FROM League
LEFT JOIN Match
USING (country_id)
-- Filter for the 2013/2014 season ตรงนี้ต้องเหมือนกันกับใน subquery
WHERE season = '2013/2014'
GROUP BY League;''', engine)

,league,avg_goals,overall_avg
0,Belgium Jupiler League,2.50,2.77
1,England Premier League,2.77,2.77
2,France Ligue 1,2.46,2.77
3,Germany 1. Bundesliga,3.16,2.77
4,Italy Serie A,2.72,2.77
5,Netherlands Eredivisie,3.20,2.77
6,Poland Ekstraklasa,2.64,2.77
7,Portugal Liga ZON Sagres,2.37,2.77
8,Scotland Premier League,2.75,2.77
9,Spain LIGA BBVA,2.75,2.77


### **Subqueries in Select for Calculations**

Subqueries in `SELECT` are a useful way to create calculated columns in a query. A subquery in `SELECT` can be treated as a <ins>single numeric value</ins> to use in your calculations. When writing queries in `SELECT`, it's important to remember that filtering the main query does not filter the subquery -- and vice versa.

In the previous exercise, you created a column to compare each league's average total goals to the overall average goals in the `2013/2014` season. In this exercise, you will add a column that directly compares these values by subtracting the overall average from the subquery.

* Select the average goals scored in a match for each league in the main query.
* Select the average goals scored in a match overall for the `2013/2014` season in the subquery.
* Subtract the subquery from the average number of goals calculated for each league.
* Filter the main query so that only games from the `2013/2014` season are included.

In [61]:
pd.read_sql_query('''
SELECT name AS league,
	   ROUND(AVG(home_team_goal + away_team_goal),2) AS avg_goals,
       ROUND(AVG(home_team_goal + away_team_goal) - (SELECT AVG(home_team_goal + away_team_goal) FROM Match WHERE season = '2013/2014'), 2) AS diff
FROM League
LEFT JOIN Match
USING(country_id)
WHERE season = '2013/2014'
GROUP BY league;''', engine)

,league,avg_goals,diff
0,Belgium Jupiler League,2.50,-0.27
1,England Premier League,2.77,0.00
2,France Ligue 1,2.46,-0.31
3,Germany 1. Bundesliga,3.16,0.39
4,Italy Serie A,2.72,-0.04
5,Netherlands Eredivisie,3.20,0.43
6,Poland Ekstraklasa,2.64,-0.13
7,Portugal Liga ZON Sagres,2.37,-0.40
8,Scotland Premier League,2.75,-0.02
9,Spain LIGA BBVA,2.75,-0.02


# **Subqueries everywhere! And best practices!**



In [62]:
pd.read_sql_query('''
SELECT country_id, name,
       ROUND(AVG(home_team_goal + away_team_goal),2) AS avg_goals,
      (SELECT ROUND(AVG(home_team_goal + away_team_goal),2) FROM Match WHERE season = '2013/2014') AS overall_avg
FROM  (SELECT country_id, home_team_goal, away_team_goal, season FROM Match WHERE home_team_goal > 5) AS matches
INNER JOIN Country ON Country.id = matches.country_id
WHERE season = '2013/2014'
GROUP BY country_id, name
HAVING (AVG(home_team_goal + away_team_goal) > (SELECT AVG(home_team_goal + away_team_goal) FROM Match WHERE season = '2013/2014'));''', engine)

,country_id,name,avg_goals,overall_avg
0,1729,England,6.80,2.77
1,7809,Germany,7.60,2.77
2,13274,Netherlands,6.33,2.77
3,15722,Poland,6.33,2.77
4,19694,Scotland,6.00,2.77
5,21518,Spain,7.33,2.77


### **ALL the subqueries EVERYWHERE**

In soccer leagues, games are played at *different <ins>stages</ins>*. Winning teams progress from one stage to the next, until they reach the final stage. In each stage, the stakes become higher than the previous one. The `Match` table includes data about the different stages that each match took place in.

In this lesson, you will build a final query across 3 exercises that will contain three subqueries -- one in the `SELECT` clause, one in the `FROM` clause, and one in the `WHERE` clause. In the final exercise, your query will extract data examining the average goals scored in each stage of a match. Does the average number of goals scored change as the stakes get higher from one stage to the next?

* Extract the average number of `home_team_goal` + `away_team_goal` in two `SELECT` subqueries.
* Calculate the average `home_team_goal` +`away_team_goal` for the specific `stage` in the main query.
* Filter both subqueries and the main query so that only data from the `2012/2013` season is included.
* Group the query by the `stage` column.

In [63]:
pd.read_sql_query('''
SELECT 
  -- Select the stage and average goals for each stage
	stage,
    ROUND(AVG(home_team_goal + away_team_goal), 2) AS avg_goals,
  -- Select the average overall goals for the 2012/2013  season
    ROUND((SELECT AVG(home_team_goal + away_team_goal) FROM match WHERE season = '2012/2013'), 2) AS overall
FROM match
-- Filter for the 2012/2013 season
WHERE season = '2012/2013'
-- Group by stage
GROUP BY stage;''', engine)

,stage,avg_goals,overall
0,1,2.68,2.77
1,2,2.65,2.77
2,3,2.83,2.77
3,4,2.80,2.77
4,5,2.61,2.77
5,6,2.78,2.77
6,7,2.69,2.77
7,8,3.09,2.77
8,9,2.70,2.77
9,10,2.96,2.77


### **Add a subquery in FROM**
In the previous exercise, you created a data set listing the average home and away goals in each match stage of the 2012/2013 match season.

In this next step, you will turn the main query into a *subquery* to extract a list of stages where the average home goals in a stage is *higher than* the overall average for home goals in a match.

* Calculate the average home goals and average away goals from the match table for each stage in the `FROM` clause subquery.
* Add a subquery to the `WHERE` clause that calculates the overall average home goals.
* Filter the main query for stages where the average home goals is higher than the overall average.
* Select the `stage` and `avg_goals` columns from the subquery into the main query.

In [64]:
pd.read_sql_query('''
SELECT 
	-- Select the stage and average goals from the subquery
	stage,
	ROUND(avg_goals, 2) AS avg_goals
FROM 
	-- Select the stage and average goals in 2012/2013
	(SELECT stage, AVG(home_team_goal + away_team_goal) AS avg_goals 
	 FROM match
	 WHERE season = '2012/2013'
	 GROUP BY stage) 
WHERE 
	-- where the average home goals in a stage is higher than the overall average for home goals in a match
	avg_goals > (SELECT AVG(home_team_goal + away_team_goal) FROM match WHERE season = '2012/2013');''', engine)

,stage,avg_goals
0,3,2.83
1,4,2.80
2,6,2.78
3,8,3.09
4,10,2.96
5,11,2.92
6,12,3.23
7,17,2.85
8,20,2.96
9,21,2.90


### **Add a subquery in SELECT**

In the previous exercise, you added a subquery to the `FROM` statement and selected the stages where the number of average goals in a stage exceeded the overall average number of goals in the 2012/2013 match season. In this final step, you will add a subquery in `SELECT` to compare the average number of goals scored in each stage to the total.

* Create a subquery in `SELECT` that yields the average goals scored in the `2012/2013` season. Name the new column `overall_avg`.
* Create a subquery in `FROM` that calculates the average goals scored in each stage during the `2012/2013` season.
* Filter the main query for stages where the average goals exceeds the overall average in `2012/2013`.

In [65]:
pd.read_sql_query('''
-- Select the stage and average goals from s
SELECT stage,
       ROUND(avg_goals, 2) AS avg_goal,
-- Select the overall average for 2012/2013
       (SELECT AVG(home_team_goal + away_team_goal) FROM match WHERE season = '2012/2013') AS overall_avg
FROM 
	-- Select the stage and average goals in 2012/2013 from match
	(SELECT stage,
            AVG(home_team_goal + away_team_goal) AS avg_goals
	 FROM match
	 WHERE season = '2012/2013'
	 GROUP BY stage)
WHERE 
	-- Filter the main query using the subquery :: ระวัง ตัว s ข้างหลัง avg_goal
	avg_goals > (SELECT AVG(home_team_goal + away_team_goal) 
                    FROM match WHERE season = '2012/2013');''' , engine)

,stage,avg_goal,overall_avg
0,3,2.83,2.772699
1,4,2.80,2.772699
2,6,2.78,2.772699
3,8,3.09,2.772699
4,10,2.96,2.772699
5,11,2.92,2.772699
6,12,3.23,2.772699
7,17,2.85,2.772699
8,20,2.96,2.772699
9,21,2.90,2.772699


# **Correlated Subquiries**

## **จากตัวอย่างที่แล้ว** 
### Which match stages tend to have a higher than average number of goals scored?

In [66]:
pd.read_sql_query('''
SELECT stage,
       ROUND(avg_goals, 2) AS avg_goal,
      (SELECT AVG(home_team_goal + away_team_goal) FROM match WHERE season = '2012/2013') AS overall_avg
FROM  
      (SELECT stage,
              AVG(home_team_goal + away_team_goal) AS avg_goals
	   FROM match
	   WHERE season = '2012/2013'
	   GROUP BY stage)
WHERE avg_goals > (SELECT AVG(home_team_goal + away_team_goal) 
                   FROM match WHERE season = '2012/2013');''' , engine)

,stage,avg_goal,overall_avg
0,3,2.83,2.772699
1,4,2.80,2.772699
2,6,2.78,2.772699
3,8,3.09,2.772699
4,10,2.96,2.772699
5,11,2.92,2.772699
6,12,3.23,2.772699
7,17,2.85,2.772699
8,20,2.96,2.772699
9,21,2.90,2.772699


* ผลลัพธ์ข้างบนนี้ สามารถทำได้ด้วยการเขียน correlated subquiries ????

In [67]:
pd.read_sql_query('''
SELECT s.stage,
       ROUND(s.avg_goals,2) AS avg_goal,
       (SELECT AVG(home_team_goal + away_team_goal)
       FROM match
       WHERE season = '2012/2013') AS overall_avg
FROM
       (SELECT stage,
               AVG(home_team_goal + away_team_goal) AS avg_goals
        FROM match
        WHERE season = '2012/2013'
        GROUP BY stage) AS s
WHERE s.avg_goals > (SELECT AVG(home_team_goal + away_team_goal)
                     FROM match AS m
                     WHERE s.stage > m.stage);''' , engine)

,stage,avg_goal,overall_avg
0,2,2.65,2.772699
1,3,2.83,2.772699
2,4,2.80,2.772699
3,6,2.78,2.772699
4,7,2.69,2.772699
5,8,3.09,2.772699
6,10,2.96,2.772699
7,11,2.92,2.772699
8,12,3.23,2.772699
9,15,2.76,2.772699


In [68]:
pd.read_sql_query('''
SELECT s.stage,
       ROUND(s.avg_goals,2) AS avg_goal,
       (SELECT AVG(home_team_goal + away_team_goal)
        FROM match
        WHERE season = '2012/2013') AS overall_avg
FROM
       (SELECT stage,
               AVG(home_team_goal + away_team_goal) AS avg_goals
        FROM match
        WHERE season = '2012/2013'
        GROUP BY stage) AS s
WHERE s.avg_goals > (SELECT AVG(home_team_goal + away_team_goal)
                     FROM match AS m
                     WHERE s.stage < m.stage);''' , engine)

,stage,avg_goal,overall_avg
0,3,2.83,2.772699
1,4,2.80,2.772699
2,6,2.78,2.772699
3,8,3.09,2.772699
4,10,2.96,2.772699
5,11,2.92,2.772699
6,12,3.23,2.772699
7,15,2.76,2.772699
8,16,2.72,2.772699
9,17,2.85,2.772699


### Have no idea why the result is incorrect.  

* The inequality value in the last subquery is somewhat strange.  When I copied the code from slides and ran it on Kaggle, it returned incorrect values.  When I changed it to "`s.stage < m.stage`", the result is closer to the uncorrelated subqueries, but still incorrect (Look at stage `15` for example.).  
* ### Perhaps because the correlated subqueries are evalueated in loops -- once for each row geneerated by the dataset
* ลองมาดูตัวอย่างที่ง่ายขึ้น สมมติอยากทราบว่า ในแต่ละประเทศ มีการทำประตูเฉลี่ยเท่าไหร่?
    * ถ้าใช้ `LEFT JOIN` เขียน ก็จะได้แบบด้านล่างนี้

In [69]:
pd.read_sql_query(''' 
SELECT name AS country,
       AVG(home_team_goal + away_team_goal) AS avg_goals
FROM country  c
LEFT JOIN match m ON c.id = m.country_id
GROUP BY country;''', engine)

,country,avg_goals
0,Belgium,2.801505
1,England,2.710526
2,France,2.443092
3,Germany,2.901552
4,Italy,2.616838
5,Netherlands,3.080882
6,Poland,2.425000
7,Portugal,2.534600
8,Scotland,2.633772
9,Spain,2.767105


## **เขียนแบบ CORRELATED SUBQUERY**

In [70]:
pd.read_sql_query('''
SELECT name AS country,
      (SELECT AVG(home_team_goal + away_team_goal) FROM match m
       WHERE m.country_id = c.id) AS avg_goals
FROM country c;''', engine)

# เขียน correlated subquery แล้ว ไม่จำเป็นต้องมี GROUP BY ต่อท้าย

,country,avg_goals
0,Belgium,2.801505
1,England,2.710526
2,France,2.443092
3,Germany,2.901552
4,Italy,2.616838
5,Netherlands,3.080882
6,Poland,2.425000
7,Portugal,2.534600
8,Scotland,2.633772
9,Spain,2.767105


### **Basic Correlated Subqueries**

Correlated subqueries are subqueries that reference one or more columns in the main query. Correlated subqueries depend on information in the main query to run, and thus, <ins>cannot be executed on their own</ins>.

Correlated subqueries are evaluated in SQL <ins>once per row of data retrieved</ins> -- a process that takes a lot more computing power and time than a simple subquery.

In this exercise, you will practice using correlated subqueries to examine matches with scores that are extreme outliers for each country -- *above 3 times the average score*!

* Select the `country_id`, `date`, `home_team_goal`, and `away_team_goal` columns in the main query.
* Complete the `AVG` value in the subquery.
* Complete the subquery column references, so that `country_id` is matched in the main and subquery.

### Correlated Subqueries ช้ามาก ใช้ 

```
(SELECT country_id,
       date,
       home_team_goal, 
       away_team_goal, 
       season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
```


### แทน `match` เพราะว่า ตรงกับจำนวนตัวอย่างที่ใช้ในดาต้าแคมพ์ รันเต็มๆ ครึ่งชั่วโมงก็ยังไม่ออก



In [71]:
pd.read_sql('''
SELECT country_id,
       date,
       home_team_goal, 
       away_team_goal
-- ใช้แทน match ตรงนี้
FROM (SELECT country_id,
       date,
       home_team_goal, 
       away_team_goal, 
       season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) AS main
WHERE home_team_goal + away_team_goal > (SELECT AVG((sub.home_team_goal + sub.away_team_goal) * 3)
                                         -- ใช้แทน match ตรงนี้
                                         FROM (SELECT country_id,
                                               date,
                                               home_team_goal, 
                                               away_team_goal, 
                                               season FROM match 
                                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) AS sub
                                         -- Join the main query to the subquery in WHERE
                                         WHERE main.country_id = sub.country_id);''', engine)

,country_id,date,home_team_goal,away_team_goal
0,1,2011-10-29 00:00:00,4,5
1,1729,2011-08-28 00:00:00,8,2
2,1729,2012-12-29 00:00:00,7,3
3,1729,2013-05-19 00:00:00,5,5
4,1729,2013-12-14 00:00:00,6,3
5,1729,2014-03-22 00:00:00,3,6
6,1729,2014-08-30 00:00:00,3,6
7,4769,2011-10-15 00:00:00,5,3
8,4769,2011-12-21 00:00:00,4,4
9,4769,2012-02-12 00:00:00,4,5


### **Correlated subquery with multiple conditions**

Correlated subqueries are useful for <ins>matching data across multiple columns</ins>. In the previous exercise, you generated a list of matches with extremely high scores for each country. In this exercise, you're going to add an additional column for matching to answer the question -- *what was the highest scoring match for each country, in each season?*

*Note: this query may take a while to load.

* Select the `country_id`, `date`, `home_team_goal`, and `away_team_goal` columns in the main query.
* Complete the subquery: Select the matches with the highest number of total goals.
* Match the subquery to the main query using `country_id` and `season`.
* Fill in the correct logical operator so that total goals equals the max goals recorded in the subquery.

### Correlated Subqueries ช้ามาก ใช้ 

```
(SELECT country_id,
       date,
       home_team_goal, 
       away_team_goal, 
       season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
```


### แทน `match` เพราะว่า ตรงกับจำนวนตัวอย่างที่ใช้ในดาต้าแคมพ์ รันเต็มๆ ครึ่งชั่วโมงก็ยังไม่ออก

In [72]:
pd.read_sql('''
SELECT country_id, date, home_team_goal, away_team_goal   
FROM (SELECT country_id,
       date,
       home_team_goal, 
       away_team_goal,
       season
       FROM match 
       WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) AS main
WHERE 
	-- Filter for matches with the highest number of goals scored
	   home_team_goal + away_team_goal = (SELECT MAX(sub.home_team_goal + sub.away_team_goal)
                                          FROM (SELECT country_id,
                                                date,
                                                home_team_goal, 
                                                away_team_goal,
                                                season
                                                FROM match 
                                                WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) AS sub
                                          WHERE main.country_id = sub.country_id
                                          AND main.season = sub.season);''', engine)

,country_id,date,home_team_goal,away_team_goal
0,1,2011-10-29 00:00:00,4,5
1,1,2012-11-17 00:00:00,2,6
2,1,2012-12-09 00:00:00,1,7
3,1,2013-01-19 00:00:00,2,6
4,1,2012-08-19 00:00:00,2,6
...,...,...,...,...
73,24558,2012-09-30 00:00:00,6,2
74,24558,2014-02-16 00:00:00,5,3
75,24558,2015-04-30 00:00:00,6,2
76,24558,2015-05-03 00:00:00,2,6


# **Nested Subqueries**

* จริงๆก็ทำไปแล้ว เพราะใช้

```
(SELECT country_id,
       date,
       home_team_goal, 
       away_team_goal, 
       season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
 ```
 ตัวอย่างด้านล่าง How much did each country's average differ from the overall average?
 

In [73]:
pd.read_sql_query(''' 
SELECT name AS country,
       ROUND(AVG(home_team_goal + away_team_goal), 4) AS avg_goals,
       ROUND(AVG(home_team_goal + away_team_goal) - (SELECT AVG(home_team_goal + away_team_goal) FROM match),4) AS avg_ddiff
FROM country  c
LEFT JOIN match m ON c.id = m.country_id
GROUP BY country;''', engine)

,country,avg_goals,avg_ddiff
0,Belgium,2.8015,0.0960
1,England,2.7105,0.0050
2,France,2.4431,-0.2624
3,Germany,2.9016,0.1960
4,Italy,2.6168,-0.0887
5,Netherlands,3.0809,0.3754
6,Poland,2.4250,-0.2805
7,Portugal,2.5346,-0.1709
8,Scotland,2.6338,-0.0718
9,Spain,2.7671,0.0616


## **Inside a subquery**

* ในแต่ละเดือน จำนวนประตูทั้งหมดที่ทำได้ แตกต่างจากจำนวนประตูทั้งหมดโดยเฉลี่ยที่ทำได้อย่างไร

## **Inner Subquery**

ใช้ `EXTRACT(MONTH from date)` ไม่ได้ใน SQLite ต้องใช้ `strftime('%m', date)` แทน

In [74]:
pd.read_sql_query('''
SELECT strftime('%m', date) AS month,
       SUM(home_team_goal + away_team_goal) AS goals
FROM match
GROUP BY month;''', engine)

,month,goals
0,01,5821
1,02,7448
2,03,7298
3,04,8145
4,05,6407
5,06,84
6,07,596
7,08,5988
8,09,7066
9,10,7120


## **Outer subquery**

หา average ของ sum

In [75]:
pd.read_sql_query('''
SELECT AVG(goals)
FROM (SELECT strftime('%m', date) AS month,
             SUM(home_team_goal + away_team_goal) AS goals
      FROM match
      GROUP BY month)''', engine)


,AVG(goals)
0,5857.25


## **Final query**

In [76]:
pd.read_sql_query('''
SELECT strftime('%m', date) AS month,
       SUM(home_team_goal + away_team_goal) AS total_goals,
       SUM(home_team_goal + away_team_goal) - (SELECT AVG(goals)
                                               FROM (SELECT strftime('%m', date) AS month,
                                                            SUM(home_team_goal + away_team_goal) AS goals
                                                     FROM match
                                                     GROUP BY month)) AS diff
FROM match
GROUP BY month
LIMIT 4;''', engine)

,month,total_goals,diff
0,01,5821,-36.25
1,02,7448,1590.75
2,03,7298,1440.75
3,04,8145,2287.75


# **Correlated Nested Subqueries**

* จำนวนประตูเฉลี่ยที่ทำได้ของแต่ละประเทศในฤดูกาลที่ `2011/2012` เป็นอย่างไร

In [77]:
pd.read_sql_query('''
SELECT name AS country,
      (SELECT AVG(home_team_goal + away_team_goal)
       FROM match m
-- Correlates with main query
       WHERE m.country_id = c.id 
-- Begin inner subquery
       AND id IN (SELECT id 
                  FROM match
                  WHERE season = '2011/2012')) AS avg_goals
FROM country c
GROUP BY country;''', engine)

,country,avg_goals
0,Belgium,2.879167
1,England,2.805263
2,France,2.515789
3,Germany,2.859477
4,Italy,2.583799
5,Netherlands,3.258170
6,Poland,2.195833
7,Portugal,2.641667
8,Scotland,2.635965
9,Spain,2.763158


## ตัวอย่างไม่เหมาะสม เพราะสามารถใช้ LEFT JOIN แบบง่ายๆได้เลย

In [78]:
pd.read_sql_query(''' 
SELECT name AS country,
       AVG(home_team_goal + away_team_goal) AS avg_goals
FROM country c
LEFT JOIN match m ON m.country_id = c.id 
WHERE season = '2011/2012'
GROUP BY country;''', engine)

,country,avg_goals
0,Belgium,2.879167
1,England,2.805263
2,France,2.515789
3,Germany,2.859477
4,Italy,2.583799
5,Netherlands,3.258170
6,Poland,2.195833
7,Portugal,2.641667
8,Scotland,2.635965
9,Spain,2.763158


### **Nested simple subqueries**

Nested subqueries can be either simple or correlated.

Just like an unnested subquery, a nested subquery's components can be executed independently of the outer query, while a correlated subquery requires both the outer and inner subquery to run and produce results.

In this exercise, you will practice creating a nested subquery to examine *the highest total number of goals in each season, overall, and during July across all seasons*.

* Complete the main query to select the `season` and the  `MAX(home_team_goal + away_team_goal)` in a match for each season. Name this `max_goals`.
* Complete the first simple subquery to select the `MAX(home_team_goal + away_team_goal)` in a `match` across all seasons. Name this `overall_max_goals`.
* Complete the nested subquery to select the `MAX(home_team_goal + away_team_goal)` in a `match` played in July across all seasons.
* Select the `MAX(home_team_goal + away_team_goal)` in the outer subquery. Name this entire subquery `july_max_goals`.

### คราวนี้ ไม่ต้องใช้ 

```
(SELECT country_id,
       date,
       home_team_goal, 
       away_team_goal, 
       season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
 ```
 
### แทน `match` เพราะมันต้องใส่หลายที่มาก ผลลัพธ์ต่างกันนิดหน่อยช่างมัน

In [79]:
pd.read_sql_query('''
SELECT season,
       MAX(home_team_goal + away_team_goal) AS max_team_goals,
      (SELECT MAX(home_team_goal + away_team_goal) FROM match) AS overall_max_team_goals,
      (SELECT MAX(home_team_goal + away_team_goal) FROM match WHERE id IN (SELECT id 
                                                                           FROM match 
                                                                           WHERE strftime('%m', date) = '07')) AS july_max_team_goals
FROM match
GROUP BY season; ''', engine)

,season,max_team_goals,overall_max_team_goals,july_max_team_goals
0,2008/2009,9,12,8
1,2009/2010,12,12,8
2,2010/2011,10,12,8
3,2011/2012,10,12,8
4,2012/2013,11,12,8
5,2013/2014,10,12,8
6,2014/2015,10,12,8
7,2015/2016,12,12,8


### **Nest a subquery in FROM**
What's the average number of matches per season where a team scored 5 or more goals? How does this differ by country?

Let's use a nested, correlated subquery to perform this operation. In the real world, you will probably find that nesting multiple subqueries is a task you don't have to perform often. In some cases, however, you may find yourself struggling to properly group by the column you want, or to calculate information requiring multiple mathematical transformations (i.e., an `AVG` of a `COUNT`).

Nesting subqueries and performing your transformations one step at a time, adding it to a subquery, and then performing the next set of transformations is often the easiest way to yield accurate information about your data. Let's get to it!

* `SELECT country_id, season, id`
* Generate a list of matches where ***at least one team*** scored ***5 or more goals***.

### ใช้ 

```
(SELECT country_id,
       id,
       home_team_goal, 
       away_team_goal, 
       season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
 ```
 
### แทน `match` 

In [80]:
pd.read_sql('''SELECT country_id, season, id 
FROM (SELECT country_id,
             id,
             home_team_goal, 
             away_team_goal, 
             season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) 
WHERE home_team_goal > 5 OR away_team_goal > 5;''', engine)

,country_id,season,id
0,1,2011/2012,791
1,1,2011/2012,854
2,1,2011/2012,878
3,1,2011/2012,991
4,1,2012/2013,1045
...,...,...,...
150,24558,2012/2013,25274
151,24558,2014/2015,25710
152,24558,2014/2015,25738
153,24558,2014/2015,25744


* Turn the query from the previous step into a subquery in the `FROM` statement.
* `COUNT` the match `id`s generated in the previous step, and group the query by `country_id` and `season`.

In [81]:
pd.read_sql_query(''' 
-- Count match ids
SELECT country_id,season, COUNT(id) AS matches
FROM (SELECT country_id, season, id
	  FROM (SELECT country_id,
                   id,
                   home_team_goal, 
                   away_team_goal, 
                   season 
            FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) 
	  WHERE home_team_goal >= 5 OR away_team_goal >= 5)
-- Group by country_id and season
GROUP BY country_id, season;''', engine)

,country_id,season,matches
0,1,2011/2012,11
1,1,2012/2013,12
2,1,2014/2015,11
3,1729,2011/2012,20
4,1729,2012/2013,15
5,1729,2013/2014,14
6,1729,2014/2015,11
7,4769,2011/2012,6
8,4769,2012/2013,5
9,4769,2013/2014,8


* Finally, declare the same query from step 2 as a subquery in `FROM` with the alias `outer_s`.
* `LEFT JOIN` it to the `country` table using the outer query's `country_id` column.
* Calculate an `AVG` number of high scoring `matches` per country in the main query.

In [82]:
pd.read_sql_query('''
SELECT name AS country,
       ROUND(AVG(outer_s.matches), 2) AS avg_seasonal_high_scores
FROM country c
LEFT JOIN (SELECT country_id, season, COUNT(id) AS matches
           FROM (SELECT country_id, season, id
	             FROM (SELECT country_id,
                              id,
                              home_team_goal, 
                              away_team_goal, 
                              season 
                        FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) 
	             WHERE home_team_goal >= 5 OR away_team_goal >= 5) AS inner_s
           GROUP BY country_id, season) AS outer_s
ON c.id = outer_s.country_id
GROUP BY country;''', engine)

,country,avg_seasonal_high_scores
0,Belgium,11.33
1,England,15.00
2,France,8.00
3,Germany,13.75
4,Italy,8.75
5,Netherlands,21.00
6,Poland,5.25
7,Portugal,7.50
8,Scotland,8.00
9,Spain,22.00


# **COMMON TABLE EXPRESSIONS**

* คือการเอา subquery มาเขียนไว้ก่อนนำหน้าในรูปแบบของ CTE
* สมมติ ด้านล่างนี้ อยากรู้ว่า ถ้าจะคัดมาเฉพาะ match ที่มีการทำประตตูทั้งหมด มากกว่า 10

## เขียนในรูปแบบ subquery

In [83]:
pd.read_sql_query('''
SELECT name AS country,
       COUNT(s.id) AS matches
FROM country c
INNER JOIN (SELECT country_id, id
            FROM match
            WHERE (home_team_goal + away_team_goal) >= 10) AS s
ON c.id = s.country_id
GROUP BY country;''', engine)

,country,matches
0,England,4
1,France,1
2,Germany,1
3,Netherlands,2
4,Scotland,1
5,Spain,5


## เขียน CTE แทน subquery

In [84]:
pd.read_sql_query('''
WITH s AS (SELECT country_id, id
           FROM match
           WHERE (home_team_goal + away_team_goal) >= 10) 

SELECT name AS country,
       COUNT(DISTINCT s.id) AS matches
FROM country c
INNER JOIN s
ON c.id = s.country_id
GROUP BY country;''', engine)

,country,matches
0,England,4
1,France,1
2,Germany,1
3,Netherlands,2
4,Scotland,1
5,Spain,5


## **Multiple CTEs**

In [85]:
pd.read_sql_query('''
WITH s1 AS (SELECT country_id, id
            FROM match
            WHERE (home_team_goal + away_team_goal) >= 10),
    s2 AS  (SELECT country_id, id
            FROM match
            WHERE (home_team_goal + away_team_goal) <= 1)
            
SELECT name AS country,
       COUNT(DISTINCT s1.id) AS high_scores_matches,
       COUNT(DISTINCT s2.id) AS low_scores_matches
FROM country AS c
INNER JOIN s1 ON c.id = s1.country_id
INNER JOIN s2 ON c.id = s2.country_id
GROUP BY country;''', engine)

,country,high_scores_matches,low_scores_matches
0,England,4,772
1,France,1,870
2,Germany,1,511
3,Netherlands,2,425
4,Scotland,1,477
5,Spain,5,760


### **Clean up with CTEs**

Above, you generated a list of countries and the number of matches in each country with more than 10 total goals. The query in that exercise utilized a subquery in the `FROM` statement in order to filter the matches before counting them in the main query. Below is the query you created:

```
SELECT name AS country,
       COUNT(sub.id) AS matches
FROM country AS c
INNER JOIN (SELECT country_id, id 
            FROM match
            WHERE (home_goal + away_goal) >= 10) AS sub
ON c.id = sub.country_id
GROUP BY country;
```


You can list one (or more) subqueries as **common table expressions** (CTEs) by *declaring* them ahead of your main query, which is an excellent tool for organizing information and placing it in a logical order.

In this exercise, let's rewrite a similar query using a CTE.

* Complete the syntax to declare your CTE.
* Select the `country_id` and `match id` from the `match` table in your CTE.
* Left join the CTE to the league table using `country_id`.

query ข้างบนได้ตามนี้ (ด้านล่างเปลี่ยนจาก `country` เป้น `league` เพราะเข้ากับแบบฝึกหัดมากกว่า)

In [86]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               id,
                               home_team_goal, 
                               away_team_goal, 
                               season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))

SELECT name AS league,
       COUNT(sub.id) AS matches
FROM league l
INNER JOIN (SELECT country_id, id 
            FROM filtered_match
            WHERE (home_team_goal + away_team_goal) >= 10) AS sub
ON l.id = sub.country_id
GROUP BY league
ORDER BY matches;''', engine)

,league,matches
0,Germany 1. Bundesliga,1
1,Netherlands Eredivisie,1
2,England Premier League,3
3,Spain LIGA BBVA,4


In [87]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               id,
                               home_team_goal, 
                               away_team_goal, 
                               season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')),

    match_list AS (SELECT country_id, id
                   FROM filtered_match
                   WHERE (home_team_goal + away_team_goal) >= 10)

SELECT name AS league,
       COUNT(match_list.id) AS matches
FROM league l
LEFT JOIN match_list ON l.id = match_list.country_id
GROUP BY name
ORDER BY matches;''', engine)

,league,matches
0,Belgium Jupiler League,0
1,France Ligue 1,0
2,Italy Serie A,0
3,Poland Ekstraklasa,0
4,Portugal Liga ZON Sagres,0
5,Scotland Premier League,0
6,Switzerland Super League,0
7,Germany 1. Bundesliga,1
8,Netherlands Eredivisie,1
9,England Premier League,3


### **Organizing with CTEs**

Previously, you modified a query based on a statement you completed in chapter 2 using common table expressions.

This time, let's expand on the exercise by looking at details about matches with very high scores using CTEs. Just like a subquery in `FROM`, you can join tables *inside* a CTE.

* Declare your CTE, where you create a list of all matches with the league name.
* Select the `league`, `date`, `home_team_goal`, and `away_team_goal` from the CTE.
* Filter the main query for matches with `10` or more goals.



In [88]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal, 
                               season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')),
    
         match_list AS (SELECT name AS league, 
     	                       date, 
  		                       home_goal, 
  		                       away_goal,
                               home_goal + away_goal AS total_goals
                        FROM filtered_match
                        LEFT JOIN league USING(country_id))

SELECT league, date, home_goal, away_goal
FROM match_list
WHERE total_goals >= 10;''', engine)

,league,date,home_goal,away_goal
0,England Premier League,2011-08-28 00:00:00,8,2
1,England Premier League,2012-12-29 00:00:00,7,3
2,England Premier League,2013-05-19 00:00:00,5,5
3,Germany 1. Bundesliga,2013-03-30 00:00:00,9,2
4,Netherlands Eredivisie,2011-11-06 00:00:00,6,4
5,Spain LIGA BBVA,2013-10-30 00:00:00,7,3
6,Spain LIGA BBVA,2015-04-05 00:00:00,9,1
7,Spain LIGA BBVA,2015-05-23 00:00:00,7,3
8,Spain LIGA BBVA,2014-09-20 00:00:00,2,8


### **CTEs with nested subqueries**

If you find yourself listing multiple subqueries in the `FROM` clause with nested statement, your query will likely become long, complex, and difficult to read.

Since many queries are written with the intention of being saved and re-run in the future, proper organization is key to a seamless workflow. Arranging subqueries as CTEs will save you time, space, and confusion in the long run!

* Declare a CTE that calculates the total goals from matches in August of the `2013/2014` season.
* `LEFT JOIN` the CTE onto the `league` table using `country_id` from the `match_list` CTE.
* Filter the list on the inner subquery to only select matches in August of the `2013/2014` season.

In [89]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal, 
                               season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')),
        match_list AS  (SELECT country_id, 
                               home_goal + away_goal AS goals
                               FROM filtered_match
  	-- Create a list of match IDs to filter data in the CTE
                               WHERE id IN (SELECT id FROM filtered_match
                                            WHERE season = '2013/2014' AND strftime('%m', date) = '08'))
-- Select the league name and average of goals in the CTE
SELECT name,
       ROUND(AVG(goals), 2)
FROM league
-- Join the CTE onto the league table
LEFT JOIN match_list USING(country_id)
GROUP BY name;''', engine)

,name,"ROUND(AVG(goals), 2)"
0,Belgium Jupiler League,NaN
1,England Premier League,2.00
2,France Ligue 1,2.03
3,Germany 1. Bundesliga,3.24
4,Italy Serie A,2.75
5,Netherlands Eredivisie,3.41
6,Poland Ekstraklasa,2.31
7,Portugal Liga ZON Sagres,3.00
8,Scotland Premier League,2.14
9,Spain LIGA BBVA,2.92


# **Get team names with a subquery**

Let's solve a problem we've encountered a few times in this course so far  
*  *How do you get both the home and away team names into one final query result*?

Out of the 4 techniques we just discussed, this can be performed using subqueries, correlated subqueries, and CTEs. Let's practice creating similar result sets using each of these 3 methods over the next 3 exercises, starting with subqueries in `FROM`.

* Create a query that `LEFT JOIN` `team` to `match` in order to get the identity of the home team. This becomes the subquery in the next step.

In [90]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
SELECT m.id, team_long_name AS hometeam
FROM filtered_match m
LEFT JOIN team t
ON m.hometeam_id = t.team_api_id
LIMIT 5;''', engine)

,id,hometeam
0,757,Oud-Heverlee Leuven
1,758,RAEC Mons
2,759,KRC Genk
3,760,KAA Gent
4,761,Sporting Lokeren


* Add a second subquery to the `FROM` statement to get the `away_team_name`, changing only the `hometeam_id`. Left join both subqueries to the `match` table on the `id` column.
* Warning: if your code is timing out, you have probably made a mistake in the `JOIN` and tried to join on the wrong fields which caused the table to be too big! Check your `ON` conditions!

In [91]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
SELECT date,
    -- Get the home and away team names
       hometeam,
       awayteam,
       home_goal,
       away_goal
FROM filtered_match m
-- Join the home subquery to the match table
LEFT JOIN (SELECT filtered_match.id, team.team_long_name AS hometeam
           FROM filtered_match
           LEFT JOIN team ON filtered_match.hometeam_id = team.team_api_id) AS home
USING(id)
-- Join the away subquery to the match table
LEFT JOIN (SELECT filtered_match.id, team.team_long_name AS awayteam
           FROM filtered_match
           LEFT JOIN team ON filtered_match.awayteam_id = team.team_api_id) AS away
USING(id);''', engine)

,date,hometeam,awayteam,home_goal,away_goal
0,2011-07-29 00:00:00,Oud-Heverlee Leuven,RSC Anderlecht,2,1
1,2011-07-30 00:00:00,RAEC Mons,Standard de Liège,1,1
2,2011-07-30 00:00:00,KRC Genk,Beerschot AC,3,1
3,2011-07-30 00:00:00,KAA Gent,KSV Cercle Brugge,0,1
4,2011-07-30 00:00:00,Sporting Lokeren,SV Zulte-Waregem,0,0
...,...,...,...,...,...
12832,2014-09-23 00:00:00,FC Basel,FC Vaduz,3,1
12833,2014-09-23 00:00:00,FC Thun,FC Luzern,3,2
12834,2014-09-24 00:00:00,FC St. Gallen,Grasshopper Club Zürich,3,0
12835,2014-09-24 00:00:00,FC Zürich,FC Aarau,0,0


### **Get team names with correlated subqueries**

Let's solve the same problem using correlated subqueries 
* *How do you get both the home and away team names into one final query result?*

This can easily be performed using correlated subqueries. But how might that impact the performance of your query? Complete the following steps and let's find out!

Please note that your query will run MUCH MUCH more slowly than the previous exercise!

* Using a correlated subquery in the `SELECT` statement, match the `team_api_id` column from `team` to the `hometeam_id` from `match`.

In [92]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT date,
      (SELECT team_long_name
       FROM team t
-- Connect the team to the match table
       WHERE t.team_api_id = m.hometeam_id) AS hometeam
FROM filtered_match m;''', engine)



,date,hometeam
0,2011-07-29 00:00:00,Oud-Heverlee Leuven
1,2011-07-30 00:00:00,RAEC Mons
2,2011-07-30 00:00:00,KRC Genk
3,2011-07-30 00:00:00,KAA Gent
4,2011-07-30 00:00:00,Sporting Lokeren
...,...,...
12832,2014-09-23 00:00:00,FC Basel
12833,2014-09-23 00:00:00,FC Thun
12834,2014-09-24 00:00:00,FC St. Gallen
12835,2014-09-24 00:00:00,FC Zürich


* Create a second correlated subquery in `SELECT`, yielding the away team's name.
* Select the home and away goal columns from `match` in the main query.

In [93]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT date,
      (SELECT team_long_name
       FROM team AS t
       WHERE t.team_api_id = m.hometeam_id) AS hometeam,
-- Connect the team to the match table
      (SELECT team_long_name
       FROM team AS t
       WHERE t.team_api_id = m.awayteam_id) AS awayteam,
    -- Select home and away goals
     home_goal,
     away_goal
FROM filtered_match m;''', engine)

,date,hometeam,awayteam,home_goal,away_goal
0,2011-07-29 00:00:00,Oud-Heverlee Leuven,RSC Anderlecht,2,1
1,2011-07-30 00:00:00,RAEC Mons,Standard de Liège,1,1
2,2011-07-30 00:00:00,KRC Genk,Beerschot AC,3,1
3,2011-07-30 00:00:00,KAA Gent,KSV Cercle Brugge,0,1
4,2011-07-30 00:00:00,Sporting Lokeren,SV Zulte-Waregem,0,0
...,...,...,...,...,...
12832,2014-09-23 00:00:00,FC Basel,FC Vaduz,3,1
12833,2014-09-23 00:00:00,FC Thun,FC Luzern,3,2
12834,2014-09-24 00:00:00,FC St. Gallen,Grasshopper Club Zürich,3,0
12835,2014-09-24 00:00:00,FC Zürich,FC Aarau,0,0


### **Get team names with CTEs**

You've now explored two methods for answering the question, 
* *How do you get both the home and away team names into one final query result?*

Let's explore the final method - common table expressions. Common table expressions are similar to the subquery method for generating results, mainly differing in syntax and the order in which information is processed.

* Select `id` from `match` and `team_long_name` from `team`. Join these two tables together on `hometeam_id` in `match` and `team_api_id` in `team`.

In [94]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT m.id, 
       team_long_name AS hometeam
FROM filtered_match m
-- Join team to match using team_api_id and hometeam_id
LEFT JOIN team t 
ON m.hometeam_id = t.team_api_id;''', engine)

,id,hometeam
0,757,Oud-Heverlee Leuven
1,758,RAEC Mons
2,759,KRC Genk
3,760,KAA Gent
4,761,Sporting Lokeren
...,...,...
12832,25795,FC Basel
12833,25796,FC Thun
12834,25797,FC St. Gallen
12835,25798,FC Zürich


* Declare the query from the previous step as a common table expression. `SELECT` everything from the CTE into the main query. ***Your results will not change at this step!***

In [95]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')),
               home AS (SELECT m.id, t.team_long_name AS hometeam
                        FROM filtered_match m
                        LEFT JOIN team t 
                        ON m.hometeam_id = t.team_api_id)
-- Select everything from home
SELECT *
FROM home;''', engine)

,id,hometeam
0,757,Oud-Heverlee Leuven
1,758,RAEC Mons
2,759,KRC Genk
3,760,KAA Gent
4,761,Sporting Lokeren
...,...,...
12832,25795,FC Basel
12833,25796,FC Thun
12834,25797,FC St. Gallen
12835,25798,FC Zürich


* Let's declare the second CTE, `away`. Join it to the first CTE on the `id` column.
* Add `date`, `home_goal`, and `away_goal` columns to the CTEs. `SELECT` them into the main query.

In [96]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')),
               home AS (SELECT m.id, date, team_long_name AS hometeam, home_goal
                        FROM filtered_match m
                        LEFT JOIN team t ON m.hometeam_id = t.team_api_id),
-- Declare and set up the away CTE
               away AS (SELECT m.id, date, team_long_name AS awayteam, away_goal
                        FROM filtered_match m
                        LEFT JOIN team t ON m.awayteam_id = t.team_api_id)
-- Select date, home_goal, and away_goal
SELECT home.date,
       home.hometeam,
       away.awayteam,
       home.home_goal,
       away.away_goal
-- Join away and home on the id column
FROM home
INNER JOIN away
USING(id);''', engine)

,date,hometeam,awayteam,home_goal,away_goal
0,2011-07-29 00:00:00,Oud-Heverlee Leuven,RSC Anderlecht,2,1
1,2011-07-30 00:00:00,RAEC Mons,Standard de Liège,1,1
2,2011-07-30 00:00:00,KRC Genk,Beerschot AC,3,1
3,2011-07-30 00:00:00,KAA Gent,KSV Cercle Brugge,0,1
4,2011-07-30 00:00:00,Sporting Lokeren,SV Zulte-Waregem,0,0
...,...,...,...,...,...
12832,2014-09-23 00:00:00,FC Basel,FC Vaduz,3,1
12833,2014-09-23 00:00:00,FC Thun,FC Luzern,3,2
12834,2014-09-24 00:00:00,FC St. Gallen,Grasshopper Club Zürich,3,0
12835,2014-09-24 00:00:00,FC Zürich,FC Aarau,0,0


## ว่ากันด้วยเรื่องของ GROUP BY

In [97]:
pd.read_sql_query('''
SELECT country_id, season, date, AVG(home_team_goal) AS avg_home
FROM match
GROUP BY country_id, season, date
Order BY country_id, season, date;''', engine)

,country_id,season,date,avg_home
0,1,2008/2009,2008-08-16 00:00:00,0.833333
1,1,2008/2009,2008-08-17 00:00:00,3.000000
2,1,2008/2009,2008-08-23 00:00:00,2.285714
3,1,2008/2009,2008-08-24 00:00:00,3.000000
4,1,2008/2009,2008-08-29 00:00:00,1.000000
...,...,...,...,...
8250,24558,2015/2016,2016-05-11 00:00:00,1.666667
8251,24558,2015/2016,2016-05-12 00:00:00,3.000000
8252,24558,2015/2016,2016-05-16 00:00:00,2.000000
8253,24558,2015/2016,2016-05-22 00:00:00,1.000000



### Query ด้านล่างรันบน SQL ปกติไม่ได้ จะเกิด error 

```
column "match.season" must appear in the GROUP BY clause or be used in an aggregate function
LINE 1: SELECT country_id, season, date, AVG(home_goal) AS avg_home
```

### ถึง SQLite จะรันได้ ก็ได้ผลออกมาผิด
* คอลัมน์ season และ date มีแค่ฤดูกาล `2008/2009`

## ต้องใช้ `WINDOW` Function!

In [98]:
pd.read_sql_query('''
SELECT country_id, season, date, AVG(home_team_goal) AS avg_home
FROM match
GROUP BY country_id
Order BY country_id, season, date;''', engine)

,country_id,season,date,avg_home
0,1,2008/2009,2008-08-17 00:00:00,1.609375
1,1729,2008/2009,2008-08-17 00:00:00,1.550987
2,4769,2008/2009,2008-08-09 00:00:00,1.402961
3,7809,2008/2009,2008-08-15 00:00:00,1.626634
4,10257,2008/2009,2008-08-31 00:00:00,1.500829
5,13274,2008/2009,2008-08-29 00:00:00,1.779820
6,15722,2008/2009,2008-08-09 00:00:00,1.394792
7,17642,2008/2009,2008-08-24 00:00:00,1.408382
8,19694,2008/2009,2008-08-09 00:00:00,1.429276
9,21518,2008/2009,2008-08-30 00:00:00,1.631250


# **What is a Window function?**

* ย้อนกลับไปคำถาม *How many goals were scored in each match in 2011/2012 and how did that compare to the average?*



In [99]:
pd.read_sql_query('''SELECT date, home_team_goal + away_team_goal AS goals, 
                            (SELECT AVG(home_team_goal + away_team_goal)
                                    FROM match
                                    WHERE season = '2011/2012') AS overall_avg
                      FROM match 
                      WHERE season = '2011/2012'
                      LIMIT 4;''', engine)

,date,goals,overall_avg
0,2011-07-29 00:00:00,3,2.71646
1,2011-07-30 00:00:00,2,2.71646
2,2011-07-30 00:00:00,4,2.71646
3,2011-07-30 00:00:00,1,2.71646


### ผลลัพธ์ข้างบนสามารถทำได้ด้วยการใช้ `OVER` statement
* `OVER()` tells SQL to pass this aggregate value over the existing result set (`2011/2012`)

In [100]:
pd.read_sql_query('''SELECT date, home_team_goal + away_team_goal AS goals, 
                            AVG(home_team_goal + away_team_goal) OVER() AS overall_avg
                      FROM match 
                      WHERE season = '2011/2012'
                      LIMIT 4;''', engine)

,date,goals,overall_avg
0,2011-07-29 00:00:00,3,2.71646
1,2011-07-30 00:00:00,2,2.71646
2,2011-07-30 00:00:00,4,2.71646
3,2011-07-30 00:00:00,1,2.71646


# **Generate a Rank**
* ตามชื่อเลย คือ สร้างคอลัมน์ใหม่แล้วจัดอันดับให้คอลัมน์ที่เราระบุ
* *What is the rank of matches based on number of goals scored?*

In [101]:
pd.read_sql_query(''' SELECT date, home_team_goal + away_team_goal AS goals 
                      FROM match 
                      WHERE season = '2011/2012'
                      LIMIT 4;''', engine)

,date,goals
0,2011-07-29 00:00:00,3
1,2011-07-30 00:00:00,2
2,2011-07-30 00:00:00,4
3,2011-07-30 00:00:00,1


### ใช้คำสั่ง `RANK() OVER(ORDER BY column) AS col_rank`

By default, `RANK()` orders from smallest to largest ถ้าจะเรียงกลับ ให้ใส่ `DESC`

ข้อควรจำ:
1. Window functions are processed <ins>***after the entire query***</ins> except the final `ORDER BY` statement.

In [102]:
pd.read_sql_query(''' SELECT date, home_team_goal + away_team_goal AS goals, 
                             RANK() OVER(ORDER BY home_team_goal + away_team_goal) AS goals_rank
                      FROM match 
                      WHERE season = '2011/2012'
                      LIMIT 4;''', engine)

,date,goals,goals_rank
0,2011-07-30 00:00:00,0,1
1,2011-11-05 00:00:00,0,1
2,2011-12-04 00:00:00,0,1
3,2011-12-04 00:00:00,0,1


In [103]:
pd.read_sql_query(''' SELECT date, home_team_goal + away_team_goal AS goals, 
                             RANK() OVER(ORDER BY home_team_goal + away_team_goal DESC) AS goals_rank
                      FROM match 
                      WHERE season = '2011/2012'
                      LIMIT 4;''', engine)

,date,goals,goals_rank
0,2011-08-28 00:00:00,10,1
1,2011-11-06 00:00:00,10,1
2,2011-10-29 00:00:00,9,3
3,2012-02-12 00:00:00,9,3


### **The match is OVER**

The `OVER()` clause allows you to pass an aggregate function down a data set, similar to subqueries in `SELECT`. The `OVER()` clause offers significant benefits over subqueries in select -- namely, your queries will run faster, and the `OVER()` clause has a wide range of additional functions and clauses you can include with it.

In this exercise, you will revise some queries from previous chapters using the `OVER()` clause.

* Select the match `id`, country `name`, `season`, `home_goal`, and `away_goal` from the `filtered_match` CTE and `country` tables.
* Complete the query that calculates the average number of goals scored overall and then includes the aggregate value in each row ***using a window function***.

In [104]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT m.id, 
       name AS country, 
       season,
	   home_goal,
	   away_goal,
    -- Use a window to include the aggregate average in each row
	   AVG(home_goal + away_goal) OVER() AS overall_avg
FROM filtered_match m
LEFT JOIN country c ON m.country_id = c.id;''', engine)

,id,country,season,home_goal,away_goal,overall_avg
0,757,Belgium,2011/2012,2,1,2.732103
1,758,Belgium,2011/2012,1,1,2.732103
2,759,Belgium,2011/2012,3,1,2.732103
3,760,Belgium,2011/2012,0,1,2.732103
4,761,Belgium,2011/2012,0,0,2.732103
...,...,...,...,...,...,...
12832,25795,Switzerland,2014/2015,3,1,2.732103
12833,25796,Switzerland,2014/2015,3,2,2.732103
12834,25797,Switzerland,2014/2015,3,0,2.732103
12835,25798,Switzerland,2014/2015,0,0,2.732103


### **What's OVER here?**

Window functions allow you to create a `RANK` of information according to any variable you want to use to sort your data. When setting this up, you will need to specify what column/calculation you want to use to calculate your rank. This is done by including an <ins>`ORDER BY`clause inside the `OVER()`</ins> clause. Below is an example:

```
SELECT id,
       RANK() OVER(ORDER BY home_goal) AS rank
FROM match;
```

In this exercise, you will create a data set of ranked matches according to which leagues, on average, score the most goals in a match.


* Select the league `name` and average total goals scored from `league` and `filtered_match` CTE.
* Complete the window function so it calculates the rank of average goals scored across all leagues in the database.
* Order the rank by the average total of home and away goals scored.

In [105]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT name AS league,
       AVG(home_goal + away_goal) AS avg_goals,
       RANK() OVER(ORDER BY AVG(home_goal + away_goal)) AS league_rank
FROM league l
LEFT JOIN filtered_match m 
ON l.id = m.country_id
WHERE season = '2011/2012'
GROUP BY league
ORDER BY league_rank;''', engine)

,league,avg_goals,league_rank
0,Poland Ekstraklasa,2.195833,1
1,France Ligue 1,2.515789,2
2,Italy Serie A,2.583799,3
3,Switzerland Super League,2.623457,4
4,Scotland Premier League,2.635965,5
5,Portugal Liga ZON Sagres,2.641667,6
6,Spain LIGA BBVA,2.763158,7
7,England Premier League,2.805263,8
8,Germany 1. Bundesliga,2.859477,9
9,Belgium Jupiler League,2.879167,10


### **Flip OVER your results**

In the last exercise, the rank generated in your query was organized from smallest to largest. By adding `DESC` to your window function, you can create a rank sorted from largest to smallest.

```
SELECT 
    id,
    RANK() OVER(ORDER BY home_goal DESC) AS rank
FROM match;
```

* Complete the window function to rank each league from highest to lowest average goals scored.
* Order the main query by the rank you just created.

In [106]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT name AS league,
       AVG(home_goal + away_goal) AS avg_goals,
       RANK() OVER(ORDER BY AVG(home_goal + away_goal) DESC) AS league_rank
FROM league l
LEFT JOIN filtered_match m 
ON l.id = m.country_id
WHERE season = '2011/2012'
GROUP BY league
ORDER BY league_rank;''', engine)

,league,avg_goals,league_rank
0,Netherlands Eredivisie,3.258170,1
1,Belgium Jupiler League,2.879167,2
2,Germany 1. Bundesliga,2.859477,3
3,England Premier League,2.805263,4
4,Spain LIGA BBVA,2.763158,5
5,Portugal Liga ZON Sagres,2.641667,6
6,Scotland Premier League,2.635965,7
7,Switzerland Super League,2.623457,8
8,Italy Serie A,2.583799,9
9,France Ligue 1,2.515789,10


# **OVER and PARTITION BY**

* Calculate separate values for different categories established in a partition
* Calculate different aggregate values within the same column, e.g.,

```
AVG(home_team_goal) OVER(PARTITION BY season)
```

## **Partition your data**

* *How many goals were scored in each match and how did that compare to the overall average?*




In [107]:
pd.read_sql_query('''SELECT date,
                            home_team_goal + away_team_goal AS goals,
                            AVG(home_team_goal + away_team_goal) OVER(PARTITION BY season) AS season_avg
                            FROM match
                            WHERE season IN ('2012/2013','2011/2012', '2013/2014');''', engine)

,date,goals,season_avg
0,2011-07-29 00:00:00,3,2.716460
1,2011-07-30 00:00:00,2,2.716460
2,2011-07-30 00:00:00,4,2.716460
3,2011-07-30 00:00:00,1,2.716460
4,2011-07-30 00:00:00,0,2.716460
...,...,...,...
9507,2013-09-26 00:00:00,3,2.766821
9508,2013-09-25 00:00:00,2,2.766821
9509,2013-09-25 00:00:00,3,2.766821
9510,2013-09-25 00:00:00,2,2.766821


## **PARTITION BY multiple columns**



In [108]:
pd.read_sql_query('''SELECT DISTINCT name,
                            season,
                            --home_team_goal + away_team_goal AS goals,
                            ROUND(AVG(home_team_goal + away_team_goal) OVER(PARTITION BY season, name), 2) AS season_country_avg
                     FROM country c
                     LEFT JOIN match m ON c.id = m.country_id
                     WHERE season IN ('2011/2012', '2014/2015');''', engine)

,name,season,season_country_avg
0,Belgium,2011/2012,2.88
1,England,2011/2012,2.81
2,France,2011/2012,2.52
3,Germany,2011/2012,2.86
4,Italy,2011/2012,2.58
5,Netherlands,2011/2012,3.26
6,Poland,2011/2012,2.20
7,Portugal,2011/2012,2.64
8,Scotland,2011/2012,2.64
9,Spain,2011/2012,2.76


### **PARTITION BY a column**

The `PARTITION BY` clause allows you to calculate separate "windows" based on columns you want to divide your results. For example, you can create a single column that calculates an overall average of goals scored for each season.

In this exercise, you will be creating a data set of games played by *Legia Warszawa* (Warsaw League), the top ranked team in Poland, and comparing their individual game performance to the overall average for that season.

Where do you see more outliers? Are they *Legia Warszawa*'s home or away games?

* Complete the two window functions that calculate the home and away goal averages. Partition the window functions by season to calculate separate averages for each season.
* Filter the query to only include matches played by Legia Warszawa, `id = 8673`.

In [109]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT date, season, home_goal, away_goal,
	   CASE WHEN hometeam_id = 8673 THEN 'home' ELSE 'away' END AS warsaw_location,
       AVG(home_goal) OVER(PARTITION BY season) AS season_homeavg,
       AVG(away_goal) OVER(PARTITION BY season) AS season_awayavg
FROM filtered_match
WHERE hometeam_id = 8673 OR awayteam_id = 8673
ORDER BY (home_goal + away_goal) DESC 
LIMIT 10;''', engine)

,date,season,home_goal,away_goal,warsaw_location,season_homeavg,season_awayavg
0,2013-09-14 00:00:00,2013/2014,3,5,away,1.766667,1.233333
1,2014-09-13 00:00:00,2014/2015,4,3,home,1.566667,1.333333
2,2013-07-20 00:00:00,2013/2014,5,1,home,1.766667,1.233333
3,2013-02-23 00:00:00,2012/2013,3,2,away,1.566667,1.133333
4,2013-06-02 00:00:00,2012/2013,5,0,home,1.566667,1.133333
5,2012-10-28 00:00:00,2012/2013,3,2,home,1.566667,1.133333
6,2013-10-20 00:00:00,2013/2014,4,1,home,1.766667,1.233333
7,2013-12-15 00:00:00,2013/2014,4,1,home,1.766667,1.233333
8,2013-09-25 00:00:00,2013/2014,2,3,away,1.766667,1.233333
9,2014-08-09 00:00:00,2014/2015,5,0,home,1.566667,1.333333


### **PARTITION BY multiple columns**

The `PARTITION BY` clause can be used to break out window averages by multiple data points (columns). You can even calculate the information you want to use to partition your data! For example, you can calculate *average goals scored by season and by country, or by the calendar year* (taken from the date column).

In this exercise, you will calculate the average number home and away goals scored *Legia Warszawa*, and their opponents, partitioned by the month in each season.

* `SELECT date, season, home_goal, away_goal, CASE WHEN hometeam_id = 8673 THEN 'home' ELSE 'away' END AS warsaw_location, ` from `filtered_match` CTE
         
* Construct two window functions partitioning the average of home and away goals by `season` and `month`.
* Filter the dataset by *Legia Warszawa*'s team ID (`8673`) so that the window calculation only includes matches involving them.

In [110]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT date, season, home_goal, away_goal,
	   CASE WHEN hometeam_id = 8673 THEN 'home' 
                                    ELSE 'away' END AS warsaw_location,
       AVG(home_goal) OVER(PARTITION BY season, STRFTIME('%m', date)) AS season_mo_home,
       AVG(away_goal) OVER(PARTITION BY season, STRFTIME('%m', date)) AS season_mo_away
FROM filtered_match
WHERE hometeam_id = 8673 OR awayteam_id = 8673
ORDER BY (home_goal + away_goal) DESC
LIMIT 10;''', engine)

,date,season,home_goal,away_goal,warsaw_location,season_mo_home,season_mo_away
0,2013-09-14 00:00:00,2013/2014,3,5,away,2.250000,2.500000
1,2014-09-13 00:00:00,2014/2015,4,3,home,2.000000,2.666667
2,2013-07-20 00:00:00,2013/2014,5,1,home,2.500000,2.000000
3,2013-02-23 00:00:00,2012/2013,3,2,away,3.000000,2.000000
4,2013-06-02 00:00:00,2012/2013,5,0,home,5.000000,0.000000
5,2012-10-28 00:00:00,2012/2013,3,2,home,1.666667,2.000000
6,2013-09-25 00:00:00,2013/2014,2,3,away,2.250000,2.500000
7,2013-10-20 00:00:00,2013/2014,4,1,home,2.250000,0.750000
8,2013-12-15 00:00:00,2013/2014,4,1,home,2.250000,0.250000
9,2014-08-09 00:00:00,2014/2015,5,0,home,2.000000,1.000000


# **Sliding Window Statement and Keywords**

`ROWS BETWEEN <start> AND <finish>` โดยที่ start หรือ finish มาจากคีย์เวิร์ดเหล่านี้

* `PRECEDING`: specify the number of rows before the current row
* `FOLLOWING`: specify the number of rows after the current row
* `UNBOUNDED PRECEDING`: specify every row since the beginnine
* `UNBOUNDED FOLOOWING`: specify every row until the end
* `CURRENT ROW`: Stop calculation at the current row

## **UNBOUNDED PRECEDING as running totals (cummulative)**

In [111]:
pd.read_sql_query('''
SELECT date,home_team_goal,away_team_goal, 
       SUM(home_team_goal) OVER(ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total
FROM match
WHERE home_team_api_id = 8456 AND season = '2011/2012';''', engine)

,date,home_team_goal,away_team_goal,running_total
0,2011-08-15 00:00:00,4,0,4
1,2011-09-10 00:00:00,3,0,7
2,2011-09-24 00:00:00,2,0,9
3,2011-10-15 00:00:00,4,1,13
4,2011-10-29 00:00:00,3,1,16
5,2011-11-19 00:00:00,3,1,19
6,2011-12-03 00:00:00,5,1,24
7,2011-12-18 00:00:00,1,0,25
8,2011-12-21 00:00:00,3,0,28
9,2012-01-03 00:00:00,3,0,31


## **PRECEDING**

* Calculate the score only the current and a previous match

In [112]:
pd.read_sql_query('''
SELECT date,home_team_goal,away_team_goal,
       SUM(home_team_goal) OVER(ORDER BY date ROWS BETWEEN 1 PRECEDING AND CURRENT ROW) AS last2
FROM match
WHERE home_team_api_id = 8456
AND season = '2011/2012';''', engine)

,date,home_team_goal,away_team_goal,last2
0,2011-08-15 00:00:00,4,0,4
1,2011-09-10 00:00:00,3,0,7
2,2011-09-24 00:00:00,2,0,5
3,2011-10-15 00:00:00,4,1,6
4,2011-10-29 00:00:00,3,1,7
5,2011-11-19 00:00:00,3,1,6
6,2011-12-03 00:00:00,5,1,8
7,2011-12-18 00:00:00,1,0,6
8,2011-12-21 00:00:00,3,0,4
9,2012-01-03 00:00:00,3,0,6


### **Slide to the left**

Sliding windows allow you to create running calculations between any two points in a window using functions such as `PRECEDING`, `FOLLOWING`, and `CURRENT ROW`. You can calculate running counts, sums, averages, and other aggregate functions between any two points you specify in the data set.

In this exercise, you will expand on the examples discussed in the video, calculating the running total of goals scored by the *FC Utrecht* when they were the home team during the `2011/2012` season. Do they score more goals at the end of the season as the home or away team?

* `SELECT date, home_goal, away_goal FROM filtered_match` CTE
* Complete the window function by:
    * Assessing the running total of home goals scored by *FC Utrecht*.
    * Assessing the running average of home goals scored.
    * Ordering both the running average and running total by `date`.

In [113]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT date, home_goal, away_goal,
       SUM(home_goal) OVER(ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total,
       AVG(home_goal) OVER(ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_avg
FROM filtered_match
WHERE hometeam_id = 9908 AND season = '2011/2012';''', engine)

,date,home_goal,away_goal,running_total,running_avg
0,2011-08-14 00:00:00,2,2,2,2.000000
1,2011-08-27 00:00:00,3,1,5,2.500000
2,2011-09-18 00:00:00,2,2,7,2.333333
3,2011-10-01 00:00:00,3,0,10,2.500000
4,2011-10-22 00:00:00,1,4,11,2.200000
5,2011-11-06 00:00:00,6,4,17,2.833333
6,2011-12-04 00:00:00,2,6,19,2.714286
7,2011-12-11 00:00:00,2,2,21,2.625000
8,2012-01-22 00:00:00,1,1,22,2.444444
9,2012-02-12 00:00:00,1,1,23,2.300000


### **Slide to the right**

Now let's see how *FC Utrecht* performs when they're the away team. You'll notice that the total for the season is at the bottom of the data set you queried. Depending on your results, this could be pretty long, and scrolling down is not very helpful.

In this exercise, you will slightly modify the query from the previous exercise by sorting the data set in ***reverse*** order and calculating a backward running total **from the** `CURRENT ROW`** to the end of the data set (earliest record).**

* Select the `date`, `home_goal`, and `away_goal` from `filtered_match` CTE
* Complete the window function by:
    * Assessing the running total of home goals scored by FC Utrecht.
    * Assessing the running average of home goals scored.
    * Ordering both the running average and running total by `date`, *descending*.

In [114]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT date, home_goal, away_goal,
       SUM(home_goal) OVER(ORDER BY date DESC ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS running_total,
       AVG(home_goal) OVER(ORDER BY date DESC ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS running_avg
FROM filtered_match
WHERE awayteam_id = 9908 AND season = '2011/2012';''', engine)

,date,home_goal,away_goal,running_total,running_avg
0,2012-05-06 00:00:00,1,3,25,1.470588
1,2012-04-21 00:00:00,0,2,24,1.500000
2,2012-04-12 00:00:00,3,0,24,1.600000
3,2012-03-25 00:00:00,3,1,21,1.500000
4,2012-03-11 00:00:00,1,1,18,1.384615
5,2012-02-26 00:00:00,1,0,17,1.416667
6,2012-02-05 00:00:00,0,2,16,1.454545
7,2012-01-28 00:00:00,2,0,16,1.600000
8,2011-12-17 00:00:00,1,0,14,1.555556
9,2011-11-25 00:00:00,2,0,13,1.625000


### **Setting up the home team CTE**

In this course, we've covered ways in which you can use `CASE` statements, subqueries, common table expressions, and window functions in your queries to structure a data set that best meets your needs. For this exercise, you will be using all of these concepts to generate a list of matches in which Manchester United was defeated during the `2014/2015` English Premier League season.

Your first task is to create the first query that <ins>filters for matches where Manchester United played as the home team</ins>. This will become a common table expression in a later exercise.

* Create a `CASE` statement that identifies each match as a win, lose, or tie for Manchester United.
* Fill out the logical operators for each `WHEN` clause in the `CASE` statement (equals, greater than, less than).
* Join the tables on `hometeam_id` from `filtered_match` CTE, and `team_api_id` from `team`.
* Filter the query to only include games from the `2014/2015` season where Manchester United was the home team.

In [115]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT m.id, team_long_name,
       CASE WHEN home_goal > away_goal THEN 'MU home Win'
	        WHEN home_goal < away_goal THEN 'MU home Loss'
            ELSE 'Tie' END AS outcome
FROM filtered_match m
-- Left join team on the home team ID and team API id
LEFT JOIN team t ON m.hometeam_id = t.team_api_id
WHERE 
	-- Filter for 2014/2015 and Manchester United as the home team
	season = '2014/2015' AND team_long_name = 'Manchester United' ORDER BY m.id;''', engine)

,id,team_long_name,outcome
0,4013,Manchester United,MU home Loss
1,4031,Manchester United,MU home Win
2,4051,Manchester United,MU home Win
3,4062,Manchester United,MU home Win
4,4085,Manchester United,MU home Win
5,4105,Manchester United,MU home Win
6,4145,Manchester United,MU home Loss
7,4164,Manchester United,MU home Win
8,4181,Manchester United,MU home Win
9,4203,Manchester United,MU home Win


### **Setting up the away team CTE**

Great job! Now that you have a query identifying the home team in a match, you will perform a similar set of steps to identify the away team. Just like the previous step, you will join the match and team tables. Each of these two queries will be declared as a Common Table Expression in the following step.

The primary difference in this query is that you will be joining the tables on `awayteam_id`, and reversing the match outcomes in the `CASE` statement.

When altering `CASE` statement logic in your own work, you can reverse either the logical condition (i.e., home_goal > away_goal) or the outcome in `THEN` -- just make sure you only reverse one of the two!

* `SELECT` match `id` and `team_long_name`
* Complete the `CASE` statement syntax.
* Fill out the logical operators identifying each match as a win, loss, or tie for *Manchester United*.
* Join the `filtered_match` CTE and `team` table on `awayteam_id`, and `team_api_id`.

In [116]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT m.id, team_long_name,
       CASE WHEN home_goal > away_goal THEN 'MU away Loss'
	        WHEN home_goal < away_goal THEN 'MU away Win'
            ELSE 'Tie' END AS outcome
FROM filtered_match m
LEFT JOIN team t ON m.awayteam_id = t.team_api_id
WHERE 
	-- Filter for 2014/2015 and Manchester United as the away team
	season = '2014/2015' AND team_long_name = 'Manchester United';''', engine)

,id,team_long_name,outcome
0,4026,Manchester United,MU away Loss
1,4039,Manchester United,MU away Win
2,4075,Manchester United,MU away Win
3,4089,Manchester United,Tie
4,4117,Manchester United,Tie
5,4126,Manchester United,Tie
6,4136,Manchester United,Tie
7,4155,Manchester United,MU away Win
8,4178,Manchester United,Tie
9,4197,Manchester United,MU away Loss


### **Putting the CTEs together**

Now that you've created the two subqueries identifying the home and away team opponents, it's time to rearrange your query with the `home` and `away` subqueries as Common Table Expressions (CTEs). You'll notice that the main query includes the phrase, `SELECT DISTINCT`. Without identifying only `DISTINCT` matches, you will return a duplicate record for each game played.

Continue building the query to extract all matches played by ***Manchester United*** in the `2014/2015` season.

* Declare the `home` and `away` CTEs before your main query.
* Join your CTEs to the `filtered_match` CTE using a `LEFT JOIN`.
* Select the relevant data from the CTEs into the main query.
* Select the `date` from `filtered_match`, team names from the CTEs, and `home_goal` and `away_goal` from `filtered_match` in the main query.

In [117]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')),
                               
-- Set up the home team CTE
     home AS (SELECT m.id, team_long_name,
	                 CASE WHEN home_goal > away_goal THEN 'MU home Win'
		                  WHEN home_goal < away_goal THEN 'MU home Loss' 
  		                  ELSE 'Tie' END AS outcome
              FROM filtered_match m
              LEFT JOIN team t ON m.hometeam_id = t.team_api_id),
-- Set up the away team CTE
     away AS (SELECT m.id, team_long_name,
	                 CASE WHEN home_goal < away_goal THEN 'MU away Win'
		                  WHEN home_goal > away_goal THEN 'MU away Loss' 
  		                  ELSE 'Tie' END AS outcome
              FROM filtered_match m
              LEFT JOIN team t ON m.awayteam_id = t.team_api_id)
-- Select team names, the date and goals
SELECT DISTINCT date,
                home.team_long_name AS home_team,
                away.team_long_name AS away_team,
                home_goal,
                away_goal
-- Join the CTEs onto the match table
FROM filtered_match m
LEFT JOIN home USING(id)
LEFT JOIN away USING(id)
WHERE m.season = '2014/2015' AND (home.team_long_name = 'Manchester United' OR away.team_long_name = 'Manchester United');''', engine)

,date,home_team,away_team,home_goal,away_goal
0,2014-08-16 00:00:00,Manchester United,Swansea City,1,2
1,2014-11-02 00:00:00,Manchester City,Manchester United,1,0
2,2014-11-08 00:00:00,Manchester United,Crystal Palace,1,0
3,2014-11-22 00:00:00,Arsenal,Manchester United,1,2
4,2014-11-29 00:00:00,Manchester United,Hull City,3,0
5,2014-12-02 00:00:00,Manchester United,Stoke City,2,1
6,2014-12-08 00:00:00,Southampton,Manchester United,1,2
7,2014-12-14 00:00:00,Manchester United,Liverpool,3,0
8,2014-12-20 00:00:00,Aston Villa,Manchester United,1,1
9,2014-12-26 00:00:00,Manchester United,Newcastle United,3,1


### **Add a window function**

Fantastic! You now have a result set that retrieves the match date, home team, away team, and the goals scored by each team. You have one final component of the question left -- *how badly did Manchester United lose in each match?*

In order to determine this, let's add a window function to the main query that ranks matches by the ***absolute value of the difference*** between `home_goal` and `away_goal`. This allows us to directly compare the difference in scores without having to consider whether Manchester United played as the home or away team!

* Set up the CTEs so that the home and away teams each have a name, ID, and score associated with them.
* Select the `date`, home team name, away team name, `home_goal`, and `away_goal`s scored in the main query.
* Rank the matches and order by the difference in scores in descending order.

In [118]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')),
    home AS (SELECT m.id, team_long_name,
	                 CASE WHEN home_goal > away_goal THEN 'MU home Win'
		                  WHEN home_goal < away_goal THEN 'MU home Loss' 
  		                  ELSE 'Tie' END AS outcome
              FROM filtered_match m
              LEFT JOIN team t ON m.hometeam_id = t.team_api_id),
    away  AS (SELECT m.id, team_long_name,
	                 CASE WHEN home_goal > away_goal THEN 'MU away Loss'
		                  WHEN home_goal < away_goal THEN 'MU away Win' 
  		                  ELSE 'Tie' END AS outcome
              FROM filtered_match m
              LEFT JOIN team t ON m.awayteam_id = t.team_api_id)

SELECT DISTINCT date,
                home.team_long_name AS home_team,
                away.team_long_name AS away_team,
                home_goal, away_goal,
                RANK() OVER(ORDER BY ABS(home_goal - away_goal) DESC) as match_rank
-- Join the CTEs onto the match table
FROM filtered_match m
LEFT JOIN home USING(id)
LEFT JOIN away USING(id)
WHERE season = '2014/2015'
AND ((home.team_long_name = 'Manchester United' AND home.outcome = 'MU home Loss') 
OR   (away.team_long_name = 'Manchester United' AND away.outcome = 'MU away Loss'));''', engine)

,date,home_team,away_team,home_goal,away_goal,match_rank
0,2015-04-26 00:00:00,Everton,Manchester United,3,0,1
1,2014-09-21 00:00:00,Leicester City,Manchester United,5,3,2
2,2014-08-16 00:00:00,Manchester United,Swansea City,1,2,3
3,2014-11-02 00:00:00,Manchester City,Manchester United,1,0,3
4,2015-01-11 00:00:00,Manchester United,Southampton,0,1,3
5,2015-02-21 00:00:00,Swansea City,Manchester United,2,1,3
6,2015-04-18 00:00:00,Chelsea,Manchester United,1,0,3
7,2015-05-02 00:00:00,Manchester United,West Bromwich Albion,0,1,3
